In [ ]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.2/36.2 MB 33.8 MB/s eta 0:00:00


In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
# upload manually with button "Choose Files"
from google.colab import files
import pandas as pd

uploaded = files.upload()
df = pd.read_excel('HOMO-LUMO-energies.xlsx')
smiles_list = df["Smiles"]

Saving HOMO-LUMO-energies.xlsx to HOMO-LUMO-energies.xlsx


# Morgan

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator
from rdkit import DataStructs

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import r2_score

# -----------------------------
# CONFIG
# -----------------------------
N_BITS = 2048
RADIUS = 2
N_SPLITS = 200
TEST_SIZE = 0.20
SEEDS = np.arange(N_SPLITS)  # seeds 0..199

# For Case 2 thresholds (63-molecule dataset)
RARE_MIN = 3
UBIQ_GT = 60

CASES = ["1", "2", "3_1024", "3_512", "4_1024", "4_512"]

INPUT_XLSX = "HOMO-LUMO-energies.xlsx"   # needs columns: Smiles, dFF
OUT_DIR = Path("ml_runs_dimred_initial_only")
OUT_DIR.mkdir(parents=True, exist_ok=True)
SUMMARY_CSV = OUT_DIR / "summary_results_initial_only.csv"

# -----------------------------
# UTILITIES
# -----------------------------
def smiles_to_morgan(smiles_list, radius=2, nBits=2048):
    gen = rdFingerprintGenerator.GetMorganGenerator(radius=radius, fpSize=nBits)
    mols = [Chem.MolFromSmiles(s) if Chem.MolFromSmiles(s) is not None else Chem.MolFromSmiles("")
            for s in smiles_list]
    X = np.zeros((len(mols), nBits), dtype=np.uint8)
    for i, m in enumerate(mols):
        fp = gen.GetFingerprint(m)
        DataStructs.ConvertToNumpyArray(fp, X[i])
    return X

def bit_presence(X):
    return (X > 0).astype(np.uint8)

def compute_freq_initial_only(X_init):
    return bit_presence(X_init).sum(axis=0)

def select_cols_by_freq(freq, rule):
    if rule == "case1":
        return (freq > 0)
    elif rule == "case2":
        return (freq > 0) & (freq >= RARE_MIN) & (freq <= UBIQ_GT)
    else:
        raise ValueError("rule must be 'case1' or 'case2'")

def apply_mask(X, mask):
    return X[:, mask]

def build_fold_map(orig_dim, new_dim):
    return np.array([j % new_dim for j in range(orig_dim)], dtype=int)

def fold_binary_OR(X, idx_map):
    new_dim = idx_map.max() + 1
    out = np.zeros((X.shape[0], new_dim), dtype=np.uint8)
    for tgt in range(new_dim):
        cols = (idx_map == tgt)
        if np.any(cols):
            out[:, tgt] = np.max(X[:, cols], axis=1)
    return out

def transform_case_initial_only(X_init, case):
    meta = {}
    orig_dim = X_init.shape[1]
    meta["orig_dim"] = orig_dim

    if case in ("1", "2"):
        freq = compute_freq_initial_only(X_init)
        rule = "case1" if case == "1" else "case2"
        mask = select_cols_by_freq(freq, rule)
        X_out = apply_mask(X_init, mask)
        meta.update({
            "rule": rule,
            "final_dim": int(mask.sum()),
            "dropped": int((~mask).sum()),
        })
        return X_out, meta

    elif case.startswith("3_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(orig_dim, new_dim)
        X_out = fold_binary_OR(X_init, idx_map)
        meta.update({
            "folded_to": new_dim,
            "final_dim": new_dim,
            "dropped": 0,
        })
        return X_out, meta

    elif case.startswith("4_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(orig_dim, new_dim)
        X_fold = fold_binary_OR(X_init, idx_map)
        freq_fold = compute_freq_initial_only(X_fold)
        mask = select_cols_by_freq(freq_fold, "case2")
        X_out = apply_mask(X_fold, mask)
        meta.update({
            "folded_to": new_dim,
            "rule": "case2_after_fold",
            "final_dim": int(mask.sum()),
            "dropped_after_fold": int((~mask).sum()),
        })
        return X_out, meta

    else:
        raise ValueError(f"Unknown case: {case}")

# -----------------------------
# LOAD DATA
# -----------------------------
df = pd.read_excel(INPUT_XLSX)
smiles = df["Smiles"].astype(str).tolist()
y = df["dFF"].values

X0 = smiles_to_morgan(smiles, radius=RADIUS, nBits=N_BITS)

# -----------------------------
# BASELINE: Train on full 2048-bit fingerprints (no reduction)
# -----------------------------
r2s = []
baseline_txt = OUT_DIR / "r2_SVRrbf_baseline.txt"
with open(baseline_txt, "w") as f:
    f.write("# seed\tR2\n")
    for seed in SEEDS:
        X_tr, X_te, y_tr, y_te = train_test_split(
            X0, y, test_size=TEST_SIZE, random_state=int(seed)
        )
        model = SVR(kernel="rbf")
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_te)
        r2 = r2_score(y_te, y_pred)
        r2s.append(r2)
        f.write(f"{int(seed)}\t{r2:.6f}\n")
baseline_mean = float(np.mean(r2s))
baseline_std = float(np.std(r2s))
print(f"Baseline (2048 bits) -> Mean R2: {baseline_mean:.6f}, Std: {baseline_std:.6f}")

# -----------------------------
# RUN CASES 1–4
# -----------------------------
summary_rows = []
summary_rows.append({
    "Case": "baseline",
    "Orig_Dim": X0.shape[1],
    "Final_Dim": X0.shape[1],
    "R2_mean": baseline_mean,
    "R2_std": baseline_std,
    "Notes": "Full 2048-bit Morgan"
})

for case in CASES:
    X_case, meta = transform_case_initial_only(X0, case)

    r2s = []
    out_txt = OUT_DIR / f"r2_SVRrbf_case{case}.txt"
    with open(out_txt, "w") as f:
        f.write("# seed\tR2\n")
        for seed in SEEDS:
            X_tr, X_te, y_tr, y_te = train_test_split(
                X_case, y, test_size=TEST_SIZE, random_state=int(seed)
            )
            model = SVR(kernel="rbf")
            model.fit(X_tr, y_tr)
            y_pred = model.predict(X_te)
            r2 = r2_score(y_te, y_pred)
            r2s.append(r2)
            f.write(f"{int(seed)}\t{r2:.6f}\n")

    r2_mean = float(np.mean(r2s))
    r2_std = float(np.std(r2s))

    summary_rows.append({
        "Case": case,
        "Orig_Dim": meta.get("orig_dim", X0.shape[1]),
        "Final_Dim": meta.get("final_dim", X_case.shape[1]),
        "R2_mean": r2_mean,
        "R2_std": r2_std,
        "Notes": {k:v for k,v in meta.items() if k not in ["orig_dim","final_dim"]}
    })

# -----------------------------
# SAVE SUMMARY
# -----------------------------
summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv(SUMMARY_CSV, index=False)

print("\n=== SVR (RBF) Summary over 200 splits ===")
print(summary_df[["Case","Orig_Dim","Final_Dim","R2_mean","R2_std"]].to_string(index=False))
print(f"\nSaved summary CSV to: {SUMMARY_CSV}")


Baseline (2048 bits) -> Mean R2: 0.141265, Std: 0.254910

=== SVR (RBF) Summary over 200 splits ===
    Case  Orig_Dim  Final_Dim  R2_mean   R2_std
baseline      2048       2048 0.141265 0.254910
       1      2048        344 0.138462 0.255285
       2      2048         87 0.189947 0.298384
  3_1024      2048       1024 0.113121 0.255248
   3_512      2048        512 0.110793 0.262404
  4_1024      2048         92 0.137222 0.300582
   4_512      2048        104 0.104035 0.290882

Saved summary CSV to: ml_runs_dimred_initial_only/summary_results_initial_only.csv


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator
from rdkit import DataStructs

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import r2_score

# -----------------------------
# CONFIG
# -----------------------------
N_BITS = 2048
RADIUS = 2
N_SPLITS = 200
TEST_SIZE = 0.20
SEEDS = np.arange(N_SPLITS)  # seeds 0..199

# For Case 2 thresholds (63-molecule dataset)
RARE_MIN = 3
UBIQ_GT = 60

CASES = ["1", "2", "3_1024", "3_512", "4_1024", "4_512"]

INPUT_XLSX = "HOMO-LUMO-energies.xlsx"   # needs columns: Smiles, dFF
OUT_DIR = Path("ml_runs_dimred_initial_only")
OUT_DIR.mkdir(parents=True, exist_ok=True)
SUMMARY_CSV = OUT_DIR / "summary_results_initial_only.csv"

# -----------------------------
# UTILITIES
# -----------------------------
def smiles_to_morgan(smiles_list, radius=2, nBits=2048):
    gen = rdFingerprintGenerator.GetMorganGenerator(radius=radius, fpSize=nBits)
    mols = [Chem.MolFromSmiles(s) if Chem.MolFromSmiles(s) is not None else Chem.MolFromSmiles("")
            for s in smiles_list]
    X = np.zeros((len(mols), nBits), dtype=np.uint8)
    for i, m in enumerate(mols):
        fp = gen.GetFingerprint(m)
        DataStructs.ConvertToNumpyArray(fp, X[i])
    return X

def bit_presence(X):
    return (X > 0).astype(np.uint8)

def compute_freq_initial_only(X_init):
    return bit_presence(X_init).sum(axis=0)

def select_cols_by_freq(freq, rule):
    if rule == "case1":
        return (freq > 0)
    elif rule == "case2":
        return (freq > 0) & (freq >= RARE_MIN) & (freq <= UBIQ_GT)
    else:
        raise ValueError("rule must be 'case1' or 'case2'")

def apply_mask(X, mask):
    return X[:, mask]

def build_fold_map(orig_dim, new_dim):
    return np.array([j % new_dim for j in range(orig_dim)], dtype=int)

def fold_binary_OR(X, idx_map):
    new_dim = idx_map.max() + 1
    out = np.zeros((X.shape[0], new_dim), dtype=np.uint8)
    for tgt in range(new_dim):
        cols = (idx_map == tgt)
        if np.any(cols):
            out[:, tgt] = np.max(X[:, cols], axis=1)
    return out

def transform_case_initial_only(X_init, case):
    meta = {}
    orig_dim = X_init.shape[1]
    meta["orig_dim"] = orig_dim

    if case in ("1", "2"):
        freq = compute_freq_initial_only(X_init)
        rule = "case1" if case == "1" else "case2"
        mask = select_cols_by_freq(freq, rule)
        X_out = apply_mask(X_init, mask)
        meta.update({
            "rule": rule,
            "final_dim": int(mask.sum()),
            "dropped": int((~mask).sum()),
        })
        return X_out, meta

    elif case.startswith("3_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(orig_dim, new_dim)
        X_out = fold_binary_OR(X_init, idx_map)
        meta.update({
            "folded_to": new_dim,
            "final_dim": new_dim,
            "dropped": 0,
        })
        return X_out, meta

    elif case.startswith("4_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(orig_dim, new_dim)
        X_fold = fold_binary_OR(X_init, idx_map)
        freq_fold = compute_freq_initial_only(X_fold)
        mask = select_cols_by_freq(freq_fold, "case2")
        X_out = apply_mask(X_fold, mask)
        meta.update({
            "folded_to": new_dim,
            "rule": "case2_after_fold",
            "final_dim": int(mask.sum()),
            "dropped_after_fold": int((~mask).sum()),
        })
        return X_out, meta

    else:
        raise ValueError(f"Unknown case: {case}")

# -----------------------------
# LOAD DATA
# -----------------------------
df = pd.read_excel(INPUT_XLSX)
smiles = df["Smiles"].astype(str).tolist()
y = df["dFF"].values

X0 = smiles_to_morgan(smiles, radius=RADIUS, nBits=N_BITS)

# -----------------------------
# BASELINE: Train on full 2048-bit fingerprints (no reduction)
# -----------------------------
r2s = []
baseline_txt = OUT_DIR / "r2_SVRlinear_baseline.txt"
with open(baseline_txt, "w") as f:
    f.write("# seed\tR2\n")
    for seed in SEEDS:
        X_tr, X_te, y_tr, y_te = train_test_split(
            X0, y, test_size=TEST_SIZE, random_state=int(seed)
        )
        model = SVR(kernel="linear")
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_te)
        r2 = r2_score(y_te, y_pred)
        r2s.append(r2)
        f.write(f"{int(seed)}\t{r2:.6f}\n")
baseline_mean = float(np.mean(r2s))
baseline_std = float(np.std(r2s))
print(f"Baseline (2048 bits) -> Mean R2: {baseline_mean:.6f}, Std: {baseline_std:.6f}")

# -----------------------------
# RUN CASES 1–4
# -----------------------------
summary_rows = []
summary_rows.append({
    "Case": "baseline",
    "Orig_Dim": X0.shape[1],
    "Final_Dim": X0.shape[1],
    "R2_mean": baseline_mean,
    "R2_std": baseline_std,
    "Notes": "Full 2048-bit Morgan"
})

for case in CASES:
    X_case, meta = transform_case_initial_only(X0, case)

    r2s = []
    out_txt = OUT_DIR / f"r2_SVRlinear_case{case}.txt"
    with open(out_txt, "w") as f:
        f.write("# seed\tR2\n")
        for seed in SEEDS:
            X_tr, X_te, y_tr, y_te = train_test_split(
                X_case, y, test_size=TEST_SIZE, random_state=int(seed)
            )
            model = SVR(kernel="linear")
            model.fit(X_tr, y_tr)
            y_pred = model.predict(X_te)
            r2 = r2_score(y_te, y_pred)
            r2s.append(r2)
            f.write(f"{int(seed)}\t{r2:.6f}\n")

    r2_mean = float(np.mean(r2s))
    r2_std = float(np.std(r2s))

    summary_rows.append({
        "Case": case,
        "Orig_Dim": meta.get("orig_dim", X0.shape[1]),
        "Final_Dim": meta.get("final_dim", X_case.shape[1]),
        "R2_mean": r2_mean,
        "R2_std": r2_std,
        "Notes": {k:v for k,v in meta.items() if k not in ["orig_dim","final_dim"]}
    })

# -----------------------------
# SAVE SUMMARY
# -----------------------------
summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv(SUMMARY_CSV, index=False)

print("\n=== SVR (RBF) Summary over 200 splits ===")
print(summary_df[["Case","Orig_Dim","Final_Dim","R2_mean","R2_std"]].to_string(index=False))
print(f"\nSaved summary CSV to: {SUMMARY_CSV}")


Baseline (2048 bits) -> Mean R2: 0.152656, Std: 0.417197

=== SVR (RBF) Summary over 200 splits ===
    Case  Orig_Dim  Final_Dim   R2_mean   R2_std
baseline      2048       2048  0.152656 0.417197
       1      2048        344  0.152656 0.417197
       2      2048         87 -0.237658 0.914480
  3_1024      2048       1024  0.062612 0.420229
   3_512      2048        512 -0.033586 0.505815
  4_1024      2048         92 -0.574159 1.003556
   4_512      2048        104 -0.686001 0.938002

Saved summary CSV to: ml_runs_dimred_initial_only/summary_results_initial_only.csv


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator
from rdkit import DataStructs

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import r2_score

# -----------------------------
# CONFIG
# -----------------------------
N_BITS = 2048
RADIUS = 2
N_SPLITS = 200
TEST_SIZE = 0.20
SEEDS = np.arange(N_SPLITS)  # seeds 0..199

# For Case 2 thresholds (63-molecule dataset)
RARE_MIN = 3
UBIQ_GT = 60

CASES = ["1", "2", "3_1024", "3_512", "4_1024", "4_512"]

INPUT_XLSX = "HOMO-LUMO-energies.xlsx"   # needs columns: Smiles, dFF
OUT_DIR = Path("ml_runs_dimred_initial_only")
OUT_DIR.mkdir(parents=True, exist_ok=True)
SUMMARY_CSV = OUT_DIR / "summary_results_initial_only.csv"

# -----------------------------
# UTILITIES
# -----------------------------
def smiles_to_morgan(smiles_list, radius=2, nBits=2048):
    gen = rdFingerprintGenerator.GetMorganGenerator(radius=radius, fpSize=nBits)
    mols = [Chem.MolFromSmiles(s) if Chem.MolFromSmiles(s) is not None else Chem.MolFromSmiles("")
            for s in smiles_list]
    X = np.zeros((len(mols), nBits), dtype=np.uint8)
    for i, m in enumerate(mols):
        fp = gen.GetFingerprint(m)
        DataStructs.ConvertToNumpyArray(fp, X[i])
    return X

def bit_presence(X):
    return (X > 0).astype(np.uint8)

def compute_freq_initial_only(X_init):
    return bit_presence(X_init).sum(axis=0)

def select_cols_by_freq(freq, rule):
    if rule == "case1":
        return (freq > 0)
    elif rule == "case2":
        return (freq > 0) & (freq >= RARE_MIN) & (freq <= UBIQ_GT)
    else:
        raise ValueError("rule must be 'case1' or 'case2'")

def apply_mask(X, mask):
    return X[:, mask]

def build_fold_map(orig_dim, new_dim):
    return np.array([j % new_dim for j in range(orig_dim)], dtype=int)

def fold_binary_OR(X, idx_map):
    new_dim = idx_map.max() + 1
    out = np.zeros((X.shape[0], new_dim), dtype=np.uint8)
    for tgt in range(new_dim):
        cols = (idx_map == tgt)
        if np.any(cols):
            out[:, tgt] = np.max(X[:, cols], axis=1)
    return out

def transform_case_initial_only(X_init, case):
    meta = {}
    orig_dim = X_init.shape[1]
    meta["orig_dim"] = orig_dim

    if case in ("1", "2"):
        freq = compute_freq_initial_only(X_init)
        rule = "case1" if case == "1" else "case2"
        mask = select_cols_by_freq(freq, rule)
        X_out = apply_mask(X_init, mask)
        meta.update({
            "rule": rule,
            "final_dim": int(mask.sum()),
            "dropped": int((~mask).sum()),
        })
        return X_out, meta

    elif case.startswith("3_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(orig_dim, new_dim)
        X_out = fold_binary_OR(X_init, idx_map)
        meta.update({
            "folded_to": new_dim,
            "final_dim": new_dim,
            "dropped": 0,
        })
        return X_out, meta

    elif case.startswith("4_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(orig_dim, new_dim)
        X_fold = fold_binary_OR(X_init, idx_map)
        freq_fold = compute_freq_initial_only(X_fold)
        mask = select_cols_by_freq(freq_fold, "case2")
        X_out = apply_mask(X_fold, mask)
        meta.update({
            "folded_to": new_dim,
            "rule": "case2_after_fold",
            "final_dim": int(mask.sum()),
            "dropped_after_fold": int((~mask).sum()),
        })
        return X_out, meta

    else:
        raise ValueError(f"Unknown case: {case}")

# -----------------------------
# LOAD DATA
# -----------------------------
df = pd.read_excel(INPUT_XLSX)
smiles = df["Smiles"].astype(str).tolist()
y = df["dFF"].values

X0 = smiles_to_morgan(smiles, radius=RADIUS, nBits=N_BITS)

# -----------------------------
# BASELINE: Train on full 2048-bit fingerprints (no reduction)
# -----------------------------
r2s = []
baseline_txt = OUT_DIR / "r2_SVRsigmoid_baseline.txt"
with open(baseline_txt, "w") as f:
    f.write("# seed\tR2\n")
    for seed in SEEDS:
        X_tr, X_te, y_tr, y_te = train_test_split(
            X0, y, test_size=TEST_SIZE, random_state=int(seed)
        )
        model = SVR(kernel="sigmoid")
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_te)
        r2 = r2_score(y_te, y_pred)
        r2s.append(r2)
        f.write(f"{int(seed)}\t{r2:.6f}\n")
baseline_mean = float(np.mean(r2s))
baseline_std = float(np.std(r2s))
print(f"Baseline (2048 bits) -> Mean R2: {baseline_mean:.6f}, Std: {baseline_std:.6f}")

# -----------------------------
# RUN CASES 1–4
# -----------------------------
summary_rows = []
summary_rows.append({
    "Case": "baseline",
    "Orig_Dim": X0.shape[1],
    "Final_Dim": X0.shape[1],
    "R2_mean": baseline_mean,
    "R2_std": baseline_std,
    "Notes": "Full 2048-bit Morgan"
})

for case in CASES:
    X_case, meta = transform_case_initial_only(X0, case)

    r2s = []
    out_txt = OUT_DIR / f"r2_SVRsigmoid_case{case}.txt"
    with open(out_txt, "w") as f:
        f.write("# seed\tR2\n")
        for seed in SEEDS:
            X_tr, X_te, y_tr, y_te = train_test_split(
                X_case, y, test_size=TEST_SIZE, random_state=int(seed)
            )
            model = SVR(kernel="sigmoid")
            model.fit(X_tr, y_tr)
            y_pred = model.predict(X_te)
            r2 = r2_score(y_te, y_pred)
            r2s.append(r2)
            f.write(f"{int(seed)}\t{r2:.6f}\n")

    r2_mean = float(np.mean(r2s))
    r2_std = float(np.std(r2s))

    summary_rows.append({
        "Case": case,
        "Orig_Dim": meta.get("orig_dim", X0.shape[1]),
        "Final_Dim": meta.get("final_dim", X_case.shape[1]),
        "R2_mean": r2_mean,
        "R2_std": r2_std,
        "Notes": {k:v for k,v in meta.items() if k not in ["orig_dim","final_dim"]}
    })

# -----------------------------
# SAVE SUMMARY
# -----------------------------
summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv(SUMMARY_CSV, index=False)

print("\n=== SVR (RBF) Summary over 200 splits ===")
print(summary_df[["Case","Orig_Dim","Final_Dim","R2_mean","R2_std"]].to_string(index=False))
print(f"\nSaved summary CSV to: {SUMMARY_CSV}")


Baseline (2048 bits) -> Mean R2: 0.263703, Std: 0.257836

=== SVR (RBF) Summary over 200 splits ===
    Case  Orig_Dim  Final_Dim  R2_mean   R2_std
baseline      2048       2048 0.263703 0.257836
       1      2048        344 0.265040 0.260669
       2      2048         87 0.259356 0.327856
  3_1024      2048       1024 0.195780 0.274115
   3_512      2048        512 0.166084 0.294891
  4_1024      2048         92 0.159021 0.347371
   4_512      2048        104 0.100642 0.351531

Saved summary CSV to: ml_runs_dimred_initial_only/summary_results_initial_only.csv


#MACCS

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

from rdkit import Chem
from rdkit.Chem import MACCSkeys
from rdkit import DataStructs

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import r2_score

# -----------------------------
# CONFIG
# -----------------------------
N_BITS = 166        # MACCS effective length (RDKit returns 167; we drop bit0)
N_SPLITS = 200
TEST_SIZE = 0.20
SEEDS = np.arange(N_SPLITS)
RARE_MIN = 3
UBIQ_GT = 60

# Cases:
# 1: drop never-seen bits
# 2: drop never-seen, rare (<3), ubiquitous (>60) bits
# 3_k: fold 166 -> k by modulo (no dropping)
# 4_k: fold, then Case-2 dropping on folded space
CASES = ["1", "2", "3_83", "3_42", "4_83", "4_42"]

INPUT_XLSX = "HOMO-LUMO-energies.xlsx"   # needs columns: Smiles, dFF
OUT_DIR = Path("ml_runs_dimred_MACCS")
OUT_DIR.mkdir(parents=True, exist_ok=True)
SUMMARY_CSV = OUT_DIR / "summary_results_MACCS.csv"

# -----------------------------
# UTILITIES
# -----------------------------
def smiles_to_maccs(smiles_list):
    mols = [Chem.MolFromSmiles(s) for s in smiles_list]
    # RDKit MACCS = 167 bits; bit 0 is unused -> slice [1:] to get 166 bits
    X = np.zeros((len(mols), 167), dtype=np.uint8)
    for i, m in enumerate(mols):
        if m is None:
            continue
        fp = MACCSkeys.GenMACCSKeys(m)  # ExplicitBitVect length 167
        DataStructs.ConvertToNumpyArray(fp, X[i])
    return X[:, 1:]  # drop the unused first bit -> shape (N, 166)

def bit_presence(X):
    return (X > 0).astype(np.uint8)

def compute_freq_initial_only(X_init):
    return bit_presence(X_init).sum(axis=0)

def select_cols_by_freq(freq, rule):
    if rule == "case1":
        return (freq > 0)
    elif rule == "case2":
        return (freq > 0) & (freq >= RARE_MIN) & (freq <= UBIQ_GT)
    else:
        raise ValueError("rule must be 'case1' or 'case2'")

def apply_mask(X, mask):
    return X[:, mask]

def build_fold_map(orig_dim, new_dim):
    return np.array([j % new_dim for j in range(orig_dim)], dtype=int)

def fold_binary_OR(X, idx_map):
    new_dim = idx_map.max() + 1
    out = np.zeros((X.shape[0], new_dim), dtype=np.uint8)
    for tgt in range(new_dim):
        cols = (idx_map == tgt)
        if np.any(cols):
            out[:, tgt] = np.max(X[:, cols], axis=1)
    return out

def transform_case_initial_only(X_init, case):
    meta = {}
    orig_dim = X_init.shape[1]
    meta["orig_dim"] = orig_dim

    if case in ("1", "2"):
        freq = compute_freq_initial_only(X_init)
        rule = "case1" if case == "1" else "case2"
        mask = select_cols_by_freq(freq, rule)
        X_out = apply_mask(X_init, mask)
        meta.update({
            "rule": rule,
            "final_dim": int(mask.sum()),
            "dropped": int((~mask).sum()),
        })
        return X_out, meta

    elif case.startswith("3_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(orig_dim, new_dim)
        X_out = fold_binary_OR(X_init, idx_map)
        meta.update({
            "folded_to": new_dim,
            "final_dim": new_dim,
            "dropped": 0,
        })
        return X_out, meta

    elif case.startswith("4_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(orig_dim, new_dim)
        X_fold = fold_binary_OR(X_init, idx_map)
        freq_fold = compute_freq_initial_only(X_fold)
        mask = select_cols_by_freq(freq_fold, "case2")
        X_out = apply_mask(X_fold, mask)
        meta.update({
            "folded_to": new_dim,
            "rule": "case2_after_fold",
            "final_dim": int(mask.sum()),
            "dropped_after_fold": int((~mask).sum()),
        })
        return X_out, meta

    else:
        raise ValueError(f"Unknown case: {case}")

# -----------------------------
# LOAD DATA
# -----------------------------
df = pd.read_excel(INPUT_XLSX)
smiles = df["Smiles"].astype(str).tolist()
y = df["dFF"].values

X0 = smiles_to_maccs(smiles)     # shape (N, 166)

# -----------------------------
# BASELINE (no reduction)
# -----------------------------
r2s = []
baseline_txt = OUT_DIR / "r2_SVRrbf_baseline.txt"
with open(baseline_txt, "w") as f:
    f.write("# seed\tR2\n")
    for seed in SEEDS:
        X_tr, X_te, y_tr, y_te = train_test_split(
            X0, y, test_size=TEST_SIZE, random_state=int(seed)
        )
        model = SVR(kernel="rbf")
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_te)
        r2 = r2_score(y_te, y_pred)
        r2s.append(r2)
        f.write(f"{int(seed)}\t{r2:.6f}\n")
baseline_mean = float(np.mean(r2s))
baseline_std = float(np.std(r2s))
print(f"MACCS Baseline (166 bits) -> Mean R2: {baseline_mean:.6f}, Std: {baseline_std:.6f}")

# -----------------------------
# RUN CASES 1–4
# -----------------------------
summary_rows = []
summary_rows.append({
    "Case": "baseline",
    "Orig_Dim": X0.shape[1],
    "Final_Dim": X0.shape[1],
    "R2_mean": baseline_mean,
    "R2_std": baseline_std,
    "Notes": "Full 166-bit MACCS (bit0 dropped)"
})

for case in CASES:
    X_case, meta = transform_case_initial_only(X0, case)

    r2s = []
    out_txt = OUT_DIR / f"r2_SVRrbf_case{case}.txt"
    with open(out_txt, "w") as f:
        f.write("# seed\tR2\n")
        for seed in SEEDS:
            X_tr, X_te, y_tr, y_te = train_test_split(
                X_case, y, test_size=TEST_SIZE, random_state=int(seed)
            )
            model = SVR(kernel="rbf")
            model.fit(X_tr, y_tr)
            y_pred = model.predict(X_te)
            r2 = r2_score(y_te, y_pred)
            r2s.append(r2)
            f.write(f"{int(seed)}\t{r2:.6f}\n")

    r2_mean = float(np.mean(r2s))
    r2_std = float(np.std(r2s))

    summary_rows.append({
        "Case": case,
        "Orig_Dim": meta.get("orig_dim", X0.shape[1]),
        "Final_Dim": meta.get("final_dim", X_case.shape[1]),
        "R2_mean": r2_mean,
        "R2_std": r2_std,
        "Notes": {k:v for k,v in meta.items() if k not in ["orig_dim","final_dim"]}
    })

# -----------------------------
# SAVE SUMMARY
# -----------------------------
summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv(SUMMARY_CSV, index=False)

print("\n=== MACCS SVR (RBF) Summary over 200 splits ===")
print(summary_df[["Case","Orig_Dim","Final_Dim","R2_mean","R2_std"]].to_string(index=False))
print(f"\nSaved summary CSV to: {SUMMARY_CSV}")


MACCS Baseline (166 bits) -> Mean R2: 0.256958, Std: 0.232958

=== MACCS SVR (RBF) Summary over 200 splits ===
    Case  Orig_Dim  Final_Dim  R2_mean   R2_std
baseline       166        166 0.256958 0.232958
       1       166        106 0.254371 0.233333
       2       166         69 0.279476 0.221913
    3_83       166         83 0.228459 0.226267
    3_42       166         42 0.263090 0.252067
    4_83       166         64 0.236713 0.225615
    4_42       166         40 0.262927 0.252096

Saved summary CSV to: ml_runs_dimred_MACCS/summary_results_MACCS.csv


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

from rdkit import Chem
from rdkit.Chem import MACCSkeys
from rdkit import DataStructs

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import r2_score

# -----------------------------
# CONFIG
# -----------------------------
N_BITS = 166        # MACCS effective length (RDKit returns 167; we drop bit0)
N_SPLITS = 200
TEST_SIZE = 0.20
SEEDS = np.arange(N_SPLITS)
RARE_MIN = 3
UBIQ_GT = 60

# Cases:
# 1: drop never-seen bits
# 2: drop never-seen, rare (<3), ubiquitous (>60) bits
# 3_k: fold 166 -> k by modulo (no dropping)
# 4_k: fold, then Case-2 dropping on folded space
CASES = ["1", "2", "3_83", "3_42", "4_83", "4_42"]

INPUT_XLSX = "HOMO-LUMO-energies.xlsx"   # needs columns: Smiles, dFF
OUT_DIR = Path("ml_runs_dimred_MACCS")
OUT_DIR.mkdir(parents=True, exist_ok=True)
SUMMARY_CSV = OUT_DIR / "summary_results_MACCS.csv"

# -----------------------------
# UTILITIES
# -----------------------------
def smiles_to_maccs(smiles_list):
    mols = [Chem.MolFromSmiles(s) for s in smiles_list]
    # RDKit MACCS = 167 bits; bit 0 is unused -> slice [1:] to get 166 bits
    X = np.zeros((len(mols), 167), dtype=np.uint8)
    for i, m in enumerate(mols):
        if m is None:
            continue
        fp = MACCSkeys.GenMACCSKeys(m)  # ExplicitBitVect length 167
        DataStructs.ConvertToNumpyArray(fp, X[i])
    return X[:, 1:]  # drop the unused first bit -> shape (N, 166)

def bit_presence(X):
    return (X > 0).astype(np.uint8)

def compute_freq_initial_only(X_init):
    return bit_presence(X_init).sum(axis=0)

def select_cols_by_freq(freq, rule):
    if rule == "case1":
        return (freq > 0)
    elif rule == "case2":
        return (freq > 0) & (freq >= RARE_MIN) & (freq <= UBIQ_GT)
    else:
        raise ValueError("rule must be 'case1' or 'case2'")

def apply_mask(X, mask):
    return X[:, mask]

def build_fold_map(orig_dim, new_dim):
    return np.array([j % new_dim for j in range(orig_dim)], dtype=int)

def fold_binary_OR(X, idx_map):
    new_dim = idx_map.max() + 1
    out = np.zeros((X.shape[0], new_dim), dtype=np.uint8)
    for tgt in range(new_dim):
        cols = (idx_map == tgt)
        if np.any(cols):
            out[:, tgt] = np.max(X[:, cols], axis=1)
    return out

def transform_case_initial_only(X_init, case):
    meta = {}
    orig_dim = X_init.shape[1]
    meta["orig_dim"] = orig_dim

    if case in ("1", "2"):
        freq = compute_freq_initial_only(X_init)
        rule = "case1" if case == "1" else "case2"
        mask = select_cols_by_freq(freq, rule)
        X_out = apply_mask(X_init, mask)
        meta.update({
            "rule": rule,
            "final_dim": int(mask.sum()),
            "dropped": int((~mask).sum()),
        })
        return X_out, meta

    elif case.startswith("3_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(orig_dim, new_dim)
        X_out = fold_binary_OR(X_init, idx_map)
        meta.update({
            "folded_to": new_dim,
            "final_dim": new_dim,
            "dropped": 0,
        })
        return X_out, meta

    elif case.startswith("4_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(orig_dim, new_dim)
        X_fold = fold_binary_OR(X_init, idx_map)
        freq_fold = compute_freq_initial_only(X_fold)
        mask = select_cols_by_freq(freq_fold, "case2")
        X_out = apply_mask(X_fold, mask)
        meta.update({
            "folded_to": new_dim,
            "rule": "case2_after_fold",
            "final_dim": int(mask.sum()),
            "dropped_after_fold": int((~mask).sum()),
        })
        return X_out, meta

    else:
        raise ValueError(f"Unknown case: {case}")

# -----------------------------
# LOAD DATA
# -----------------------------
df = pd.read_excel(INPUT_XLSX)
smiles = df["Smiles"].astype(str).tolist()
y = df["dFF"].values

X0 = smiles_to_maccs(smiles)     # shape (N, 166)

# -----------------------------
# BASELINE (no reduction)
# -----------------------------
r2s = []
baseline_txt = OUT_DIR / "r2_SVRlinear_baseline.txt"
with open(baseline_txt, "w") as f:
    f.write("# seed\tR2\n")
    for seed in SEEDS:
        X_tr, X_te, y_tr, y_te = train_test_split(
            X0, y, test_size=TEST_SIZE, random_state=int(seed)
        )
        model = SVR(kernel="linear")
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_te)
        r2 = r2_score(y_te, y_pred)
        r2s.append(r2)
        f.write(f"{int(seed)}\t{r2:.6f}\n")
baseline_mean = float(np.mean(r2s))
baseline_std = float(np.std(r2s))
print(f"MACCS Baseline (166 bits) -> Mean R2: {baseline_mean:.6f}, Std: {baseline_std:.6f}")

# -----------------------------
# RUN CASES 1–4
# -----------------------------
summary_rows = []
summary_rows.append({
    "Case": "baseline",
    "Orig_Dim": X0.shape[1],
    "Final_Dim": X0.shape[1],
    "R2_mean": baseline_mean,
    "R2_std": baseline_std,
    "Notes": "Full 166-bit MACCS (bit0 dropped)"
})

for case in CASES:
    X_case, meta = transform_case_initial_only(X0, case)

    r2s = []
    out_txt = OUT_DIR / f"r2_SVRlinear_case{case}.txt"
    with open(out_txt, "w") as f:
        f.write("# seed\tR2\n")
        for seed in SEEDS:
            X_tr, X_te, y_tr, y_te = train_test_split(
                X_case, y, test_size=TEST_SIZE, random_state=int(seed)
            )
            model = SVR(kernel="linear")
            model.fit(X_tr, y_tr)
            y_pred = model.predict(X_te)
            r2 = r2_score(y_te, y_pred)
            r2s.append(r2)
            f.write(f"{int(seed)}\t{r2:.6f}\n")

    r2_mean = float(np.mean(r2s))
    r2_std = float(np.std(r2s))

    summary_rows.append({
        "Case": case,
        "Orig_Dim": meta.get("orig_dim", X0.shape[1]),
        "Final_Dim": meta.get("final_dim", X_case.shape[1]),
        "R2_mean": r2_mean,
        "R2_std": r2_std,
        "Notes": {k:v for k,v in meta.items() if k not in ["orig_dim","final_dim"]}
    })

# -----------------------------
# SAVE SUMMARY
# -----------------------------
summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv(SUMMARY_CSV, index=False)

print("\n=== MACCS SVR (RBF) Summary over 200 splits ===")
print(summary_df[["Case","Orig_Dim","Final_Dim","R2_mean","R2_std"]].to_string(index=False))
print(f"\nSaved summary CSV to: {SUMMARY_CSV}")


MACCS Baseline (166 bits) -> Mean R2: -0.548034, Std: 0.942605

=== MACCS SVR (RBF) Summary over 200 splits ===
    Case  Orig_Dim  Final_Dim   R2_mean   R2_std
baseline       166        166 -0.548034 0.942605
       1       166        106 -0.548034 0.942605
       2       166         69 -0.489863 0.919480
    3_83       166         83 -0.543728 0.815668
    3_42       166         42 -0.716796 0.961797
    4_83       166         64 -0.487042 0.776770
    4_42       166         40 -0.687250 0.963346

Saved summary CSV to: ml_runs_dimred_MACCS/summary_results_MACCS.csv


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

from rdkit import Chem
from rdkit.Chem import MACCSkeys
from rdkit import DataStructs

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import r2_score

# -----------------------------
# CONFIG
# -----------------------------
N_BITS = 166        # MACCS effective length (RDKit returns 167; we drop bit0)
N_SPLITS = 200
TEST_SIZE = 0.20
SEEDS = np.arange(N_SPLITS)
RARE_MIN = 3
UBIQ_GT = 60

# Cases:
# 1: drop never-seen bits
# 2: drop never-seen, rare (<3), ubiquitous (>60) bits
# 3_k: fold 166 -> k by modulo (no dropping)
# 4_k: fold, then Case-2 dropping on folded space
CASES = ["1", "2", "3_83", "3_42", "4_83", "4_42"]

INPUT_XLSX = "HOMO-LUMO-energies.xlsx"   # needs columns: Smiles, dFF
OUT_DIR = Path("ml_runs_dimred_MACCS")
OUT_DIR.mkdir(parents=True, exist_ok=True)
SUMMARY_CSV = OUT_DIR / "summary_results_MACCS.csv"

# -----------------------------
# UTILITIES
# -----------------------------
def smiles_to_maccs(smiles_list):
    mols = [Chem.MolFromSmiles(s) for s in smiles_list]
    # RDKit MACCS = 167 bits; bit 0 is unused -> slice [1:] to get 166 bits
    X = np.zeros((len(mols), 167), dtype=np.uint8)
    for i, m in enumerate(mols):
        if m is None:
            continue
        fp = MACCSkeys.GenMACCSKeys(m)  # ExplicitBitVect length 167
        DataStructs.ConvertToNumpyArray(fp, X[i])
    return X[:, 1:]  # drop the unused first bit -> shape (N, 166)

def bit_presence(X):
    return (X > 0).astype(np.uint8)

def compute_freq_initial_only(X_init):
    return bit_presence(X_init).sum(axis=0)

def select_cols_by_freq(freq, rule):
    if rule == "case1":
        return (freq > 0)
    elif rule == "case2":
        return (freq > 0) & (freq >= RARE_MIN) & (freq <= UBIQ_GT)
    else:
        raise ValueError("rule must be 'case1' or 'case2'")

def apply_mask(X, mask):
    return X[:, mask]

def build_fold_map(orig_dim, new_dim):
    return np.array([j % new_dim for j in range(orig_dim)], dtype=int)

def fold_binary_OR(X, idx_map):
    new_dim = idx_map.max() + 1
    out = np.zeros((X.shape[0], new_dim), dtype=np.uint8)
    for tgt in range(new_dim):
        cols = (idx_map == tgt)
        if np.any(cols):
            out[:, tgt] = np.max(X[:, cols], axis=1)
    return out

def transform_case_initial_only(X_init, case):
    meta = {}
    orig_dim = X_init.shape[1]
    meta["orig_dim"] = orig_dim

    if case in ("1", "2"):
        freq = compute_freq_initial_only(X_init)
        rule = "case1" if case == "1" else "case2"
        mask = select_cols_by_freq(freq, rule)
        X_out = apply_mask(X_init, mask)
        meta.update({
            "rule": rule,
            "final_dim": int(mask.sum()),
            "dropped": int((~mask).sum()),
        })
        return X_out, meta

    elif case.startswith("3_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(orig_dim, new_dim)
        X_out = fold_binary_OR(X_init, idx_map)
        meta.update({
            "folded_to": new_dim,
            "final_dim": new_dim,
            "dropped": 0,
        })
        return X_out, meta

    elif case.startswith("4_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(orig_dim, new_dim)
        X_fold = fold_binary_OR(X_init, idx_map)
        freq_fold = compute_freq_initial_only(X_fold)
        mask = select_cols_by_freq(freq_fold, "case2")
        X_out = apply_mask(X_fold, mask)
        meta.update({
            "folded_to": new_dim,
            "rule": "case2_after_fold",
            "final_dim": int(mask.sum()),
            "dropped_after_fold": int((~mask).sum()),
        })
        return X_out, meta

    else:
        raise ValueError(f"Unknown case: {case}")

# -----------------------------
# LOAD DATA
# -----------------------------
df = pd.read_excel(INPUT_XLSX)
smiles = df["Smiles"].astype(str).tolist()
y = df["dFF"].values

X0 = smiles_to_maccs(smiles)     # shape (N, 166)

# -----------------------------
# BASELINE (no reduction)
# -----------------------------
r2s = []
baseline_txt = OUT_DIR / "r2_SVRsigmoid_baseline.txt"
with open(baseline_txt, "w") as f:
    f.write("# seed\tR2\n")
    for seed in SEEDS:
        X_tr, X_te, y_tr, y_te = train_test_split(
            X0, y, test_size=TEST_SIZE, random_state=int(seed)
        )
        model = SVR(kernel="sigmoid")
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_te)
        r2 = r2_score(y_te, y_pred)
        r2s.append(r2)
        f.write(f"{int(seed)}\t{r2:.6f}\n")
baseline_mean = float(np.mean(r2s))
baseline_std = float(np.std(r2s))
print(f"MACCS Baseline (166 bits) -> Mean R2: {baseline_mean:.6f}, Std: {baseline_std:.6f}")

# -----------------------------
# RUN CASES 1–4
# -----------------------------
summary_rows = []
summary_rows.append({
    "Case": "baseline",
    "Orig_Dim": X0.shape[1],
    "Final_Dim": X0.shape[1],
    "R2_mean": baseline_mean,
    "R2_std": baseline_std,
    "Notes": "Full 166-bit MACCS (bit0 dropped)"
})

for case in CASES:
    X_case, meta = transform_case_initial_only(X0, case)

    r2s = []
    out_txt = OUT_DIR / f"r2_SVRsigmoid_case{case}.txt"
    with open(out_txt, "w") as f:
        f.write("# seed\tR2\n")
        for seed in SEEDS:
            X_tr, X_te, y_tr, y_te = train_test_split(
                X_case, y, test_size=TEST_SIZE, random_state=int(seed)
            )
            model = SVR(kernel="sigmoid")
            model.fit(X_tr, y_tr)
            y_pred = model.predict(X_te)
            r2 = r2_score(y_te, y_pred)
            r2s.append(r2)
            f.write(f"{int(seed)}\t{r2:.6f}\n")

    r2_mean = float(np.mean(r2s))
    r2_std = float(np.std(r2s))

    summary_rows.append({
        "Case": case,
        "Orig_Dim": meta.get("orig_dim", X0.shape[1]),
        "Final_Dim": meta.get("final_dim", X_case.shape[1]),
        "R2_mean": r2_mean,
        "R2_std": r2_std,
        "Notes": {k:v for k,v in meta.items() if k not in ["orig_dim","final_dim"]}
    })

# -----------------------------
# SAVE SUMMARY
# -----------------------------
summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv(SUMMARY_CSV, index=False)

print("\n=== MACCS SVR (SIGMOID) Summary over 200 splits ===")
print(summary_df[["Case","Orig_Dim","Final_Dim","R2_mean","R2_std"]].to_string(index=False))
print(f"\nSaved summary CSV to: {SUMMARY_CSV}")


MACCS Baseline (166 bits) -> Mean R2: 0.216085, Std: 0.231172

=== MACCS SVR (SIGMOID) Summary over 200 splits ===
    Case  Orig_Dim  Final_Dim   R2_mean   R2_std
baseline       166        166  0.216085 0.231172
       1       166        106  0.195889 0.237163
       2       166         69  0.167250 0.246002
    3_83       166         83  0.043152 0.245977
    3_42       166         42 -0.279763 0.298899
    4_83       166         64 -0.024914 0.273245
    4_42       166         40 -0.346368 0.326515

Saved summary CSV to: ml_runs_dimred_MACCS/summary_results_MACCS.csv


# Daylight

In [ ]:
# If in Colab, first:  !pip -q install rdkit openpyxl

import numpy as np
import pandas as pd
from pathlib import Path

from rdkit import Chem
from rdkit import DataStructs

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import r2_score

# -----------------------------
# CONFIG
# -----------------------------
N_BITS = 2048                 # Daylight-like FP size
MIN_PATH = 1
MAX_PATH = 7
BRANCHED_PATHS = True

N_SPLITS = 200
TEST_SIZE = 0.20
SEEDS = np.arange(N_SPLITS)   # 0..199

# Case-2 thresholds for your ~63-molecule dataset
RARE_MIN = 3
UBIQ_GT = 60

# Cases:
# 1: drop never-seen bits
# 2: drop never-seen, rare (<3), ubiquitous (>60)
# 3_k: fold 2048 -> k by modulo (no dropping)
# 4_k: fold, then Case-2 dropping on folded space
CASES = ["1", "2", "3_1024", "3_512", "4_1024", "4_512"]

INPUT_XLSX = "HOMO-LUMO-energies.xlsx"   # must have columns: Smiles, dFF
OUT_DIR = Path("ml_runs_dimred_RDKFP")
OUT_DIR.mkdir(parents=True, exist_ok=True)
SUMMARY_CSV = OUT_DIR / "summary_results_RDKFP.csv"

# -----------------------------
# UTILITIES
# -----------------------------
def smiles_to_daylight(smiles_list, fpSize=N_BITS, minPath=MIN_PATH, maxPath=MAX_PATH, branchedPaths=BRANCHED_PATHS):
    """Classic RDKit Daylight-like (path) fingerprint -> numpy array (N, fpSize)."""
    mols = [Chem.MolFromSmiles(s) for s in smiles_list]
    fps = [Chem.RDKFingerprint(m, fpSize=fpSize, minPath=minPath, maxPath=maxPath,
                               branchedPaths=branchedPaths) if m is not None else None
           for m in mols]
    X = np.zeros((len(fps), fpSize), dtype=np.uint8)
    for i, fp in enumerate(fps):
        if fp is None:
            continue
        DataStructs.ConvertToNumpyArray(fp, X[i])
    return X

def bit_presence(X):
    return (X > 0).astype(np.uint8)

def compute_freq_initial_only(X_init):
    # count in how many molecules each bit is present
    return bit_presence(X_init).sum(axis=0)

def select_cols_by_freq(freq, rule):
    if rule == "case1":        # drop never-seen
        return (freq > 0)
    elif rule == "case2":      # drop never-seen, rare (<3), ubiquitous (>60)
        return (freq > 0) & (freq >= RARE_MIN) & (freq <= UBIQ_GT)
    else:
        raise ValueError("rule must be 'case1' or 'case2'")

def apply_mask(X, mask):
    return X[:, mask]

def build_fold_map(orig_dim, new_dim):
    # modulo hashing: original column j -> j % new_dim
    return np.array([j % new_dim for j in range(orig_dim)], dtype=int)

def fold_binary_OR(X, idx_map):
    """Fold columns into new_dim buckets using OR aggregation."""
    new_dim = int(idx_map.max()) + 1
    out = np.zeros((X.shape[0], new_dim), dtype=np.uint8)
    for tgt in range(new_dim):
        cols = (idx_map == tgt)
        if np.any(cols):
            out[:, tgt] = np.max(X[:, cols], axis=1)
    return out

def transform_case_initial_only(X_init, case):
    """
    Case '1': drop never-seen bits
    Case '2': drop never-seen, rare (<3), ubiquitous (>60)
    Case '3_k': fold to k (no dropping)
    Case '4_k': fold to k, then Case-2 dropping on folded space
    """
    meta = {"orig_dim": X_init.shape[1]}
    if case in ("1", "2"):
        freq = compute_freq_initial_only(X_init)
        rule = "case1" if case == "1" else "case2"
        mask = select_cols_by_freq(freq, rule)
        X_out = apply_mask(X_init, mask)
        meta.update({
            "rule": rule,
            "final_dim": int(mask.sum()),
            "dropped": int((~mask).sum()),
        })
        return X_out, meta

    elif case.startswith("3_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(X_init.shape[1], new_dim)
        X_out = fold_binary_OR(X_init, idx_map)
        meta.update({
            "folded_to": new_dim,
            "final_dim": new_dim,
            "dropped": 0,
        })
        return X_out, meta

    elif case.startswith("4_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(X_init.shape[1], new_dim)
        X_fold = fold_binary_OR(X_init, idx_map)
        freq_fold = compute_freq_initial_only(X_fold)
        mask = select_cols_by_freq(freq_fold, "case2")
        X_out = apply_mask(X_fold, mask)
        meta.update({
            "folded_to": new_dim,
            "rule": "case2_after_fold",
            "final_dim": int(mask.sum()),
            "dropped_after_fold": int((~mask).sum()),
        })
        return X_out, meta

    else:
        raise ValueError(f"Unknown case: {case}")

# -----------------------------
# LOAD DATA & BUILD FEATURES
# -----------------------------
df = pd.read_excel(INPUT_XLSX)
smiles = df["Smiles"].astype(str).tolist()
y = df["dFF"].values

X0 = smiles_to_daylight(smiles)   # shape (N, 2048)

# -----------------------------
# BASELINE: full 2048-bit Daylight FP
# -----------------------------
r2s = []
baseline_txt = OUT_DIR / "r2_SVRrbf_baseline.txt"
with open(baseline_txt, "w") as f:
    f.write("# seed\tR2\n")
    for seed in SEEDS:
        X_tr, X_te, y_tr, y_te = train_test_split(
            X0, y, test_size=TEST_SIZE, random_state=int(seed)
        )
        model = SVR(kernel="rbf")
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_te)
        r2 = r2_score(y_te, y_pred)
        r2s.append(r2)
        f.write(f"{int(seed)}\t{r2:.6f}\n")
baseline_mean = float(np.mean(r2s))
baseline_std  = float(np.std(r2s))
print(f"Daylight Baseline (2048 bits) -> Mean R2: {baseline_mean:.6f}, Std: {baseline_std:.6f}")

# -----------------------------
# RUN CASES 1–4
# -----------------------------
summary_rows = [{
    "Case": "baseline",
    "Orig_Dim": X0.shape[1],
    "Final_Dim": X0.shape[1],
    "R2_mean": baseline_mean,
    "R2_std": baseline_std,
    "Notes": "Full 2048-bit RDKit (Daylight-like) FP"
}]

for case in CASES:
    X_case, meta = transform_case_initial_only(X0, case)

    r2s = []
    out_txt = OUT_DIR / f"r2_SVRrbf_case{case}.txt"
    with open(out_txt, "w") as f:
        f.write("# seed\tR2\n")
        for seed in SEEDS:
            X_tr, X_te, y_tr, y_te = train_test_split(
                X_case, y, test_size=TEST_SIZE, random_state=int(seed)
            )
            model = SVR(kernel="rbf")
            model.fit(X_tr, y_tr)
            y_pred = model.predict(X_te)
            r2 = r2_score(y_te, y_pred)
            r2s.append(r2)
            f.write(f"{int(seed)}\t{r2:.6f}\n")

    r2_mean = float(np.mean(r2s))
    r2_std  = float(np.std(r2s))

    summary_rows.append({
        "Case": case,
        "Orig_Dim": meta.get("orig_dim", X0.shape[1]),
        "Final_Dim": meta.get("final_dim", X_case.shape[1]),
        "R2_mean": r2_mean,
        "R2_std": r2_std,
        "Notes": {k:v for k,v in meta.items() if k not in ["orig_dim","final_dim"]}
    })

# -----------------------------
# SAVE SUMMARY
# -----------------------------
summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv(SUMMARY_CSV, index=False)

print("\n=== RDKit (Daylight-like) SVR (RBF) Summary over 200 splits ===")
print(summary_df[["Case","Orig_Dim","Final_Dim","R2_mean","R2_std"]].to_string(index=False))
print(f"\nSaved summary CSV to: {SUMMARY_CSV}")


Daylight Baseline (2048 bits) -> Mean R2: 0.118657, Std: 0.264341

=== RDKit (Daylight-like) SVR (RBF) Summary over 200 splits ===
    Case  Orig_Dim  Final_Dim  R2_mean   R2_std
baseline      2048       2048 0.118657 0.264341
       1      2048       1840 0.117970 0.264966
       2      2048       1202 0.132650 0.253330
  3_1024      2048       1024 0.101384 0.263624
   3_512      2048        512 0.110711 0.279029
  4_1024      2048        909 0.104545 0.261213
   4_512      2048        510 0.111133 0.279004

Saved summary CSV to: ml_runs_dimred_RDKFP/summary_results_RDKFP.csv


In [ ]:
# If in Colab, first:  !pip -q install rdkit openpyxl

import numpy as np
import pandas as pd
from pathlib import Path

from rdkit import Chem
from rdkit import DataStructs

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import r2_score

# -----------------------------
# CONFIG
# -----------------------------
N_BITS = 2048                 # Daylight-like FP size
MIN_PATH = 1
MAX_PATH = 7
BRANCHED_PATHS = True

N_SPLITS = 200
TEST_SIZE = 0.20
SEEDS = np.arange(N_SPLITS)   # 0..199

# Case-2 thresholds for your ~63-molecule dataset
RARE_MIN = 3
UBIQ_GT = 60

# Cases:
# 1: drop never-seen bits
# 2: drop never-seen, rare (<3), ubiquitous (>60)
# 3_k: fold 2048 -> k by modulo (no dropping)
# 4_k: fold, then Case-2 dropping on folded space
CASES = ["1", "2", "3_1024", "3_512", "4_1024", "4_512"]

INPUT_XLSX = "HOMO-LUMO-energies.xlsx"   # must have columns: Smiles, dFF
OUT_DIR = Path("ml_runs_dimred_RDKFP")
OUT_DIR.mkdir(parents=True, exist_ok=True)
SUMMARY_CSV = OUT_DIR / "summary_results_RDKFP.csv"

# -----------------------------
# UTILITIES
# -----------------------------
def smiles_to_daylight(smiles_list, fpSize=N_BITS, minPath=MIN_PATH, maxPath=MAX_PATH, branchedPaths=BRANCHED_PATHS):
    """Classic RDKit Daylight-like (path) fingerprint -> numpy array (N, fpSize)."""
    mols = [Chem.MolFromSmiles(s) for s in smiles_list]
    fps = [Chem.RDKFingerprint(m, fpSize=fpSize, minPath=minPath, maxPath=maxPath,
                               branchedPaths=branchedPaths) if m is not None else None
           for m in mols]
    X = np.zeros((len(fps), fpSize), dtype=np.uint8)
    for i, fp in enumerate(fps):
        if fp is None:
            continue
        DataStructs.ConvertToNumpyArray(fp, X[i])
    return X

def bit_presence(X):
    return (X > 0).astype(np.uint8)

def compute_freq_initial_only(X_init):
    # count in how many molecules each bit is present
    return bit_presence(X_init).sum(axis=0)

def select_cols_by_freq(freq, rule):
    if rule == "case1":        # drop never-seen
        return (freq > 0)
    elif rule == "case2":      # drop never-seen, rare (<3), ubiquitous (>60)
        return (freq > 0) & (freq >= RARE_MIN) & (freq <= UBIQ_GT)
    else:
        raise ValueError("rule must be 'case1' or 'case2'")

def apply_mask(X, mask):
    return X[:, mask]

def build_fold_map(orig_dim, new_dim):
    # modulo hashing: original column j -> j % new_dim
    return np.array([j % new_dim for j in range(orig_dim)], dtype=int)

def fold_binary_OR(X, idx_map):
    """Fold columns into new_dim buckets using OR aggregation."""
    new_dim = int(idx_map.max()) + 1
    out = np.zeros((X.shape[0], new_dim), dtype=np.uint8)
    for tgt in range(new_dim):
        cols = (idx_map == tgt)
        if np.any(cols):
            out[:, tgt] = np.max(X[:, cols], axis=1)
    return out

def transform_case_initial_only(X_init, case):
    """
    Case '1': drop never-seen bits
    Case '2': drop never-seen, rare (<3), ubiquitous (>60)
    Case '3_k': fold to k (no dropping)
    Case '4_k': fold to k, then Case-2 dropping on folded space
    """
    meta = {"orig_dim": X_init.shape[1]}
    if case in ("1", "2"):
        freq = compute_freq_initial_only(X_init)
        rule = "case1" if case == "1" else "case2"
        mask = select_cols_by_freq(freq, rule)
        X_out = apply_mask(X_init, mask)
        meta.update({
            "rule": rule,
            "final_dim": int(mask.sum()),
            "dropped": int((~mask).sum()),
        })
        return X_out, meta

    elif case.startswith("3_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(X_init.shape[1], new_dim)
        X_out = fold_binary_OR(X_init, idx_map)
        meta.update({
            "folded_to": new_dim,
            "final_dim": new_dim,
            "dropped": 0,
        })
        return X_out, meta

    elif case.startswith("4_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(X_init.shape[1], new_dim)
        X_fold = fold_binary_OR(X_init, idx_map)
        freq_fold = compute_freq_initial_only(X_fold)
        mask = select_cols_by_freq(freq_fold, "case2")
        X_out = apply_mask(X_fold, mask)
        meta.update({
            "folded_to": new_dim,
            "rule": "case2_after_fold",
            "final_dim": int(mask.sum()),
            "dropped_after_fold": int((~mask).sum()),
        })
        return X_out, meta

    else:
        raise ValueError(f"Unknown case: {case}")

# -----------------------------
# LOAD DATA & BUILD FEATURES
# -----------------------------
df = pd.read_excel(INPUT_XLSX)
smiles = df["Smiles"].astype(str).tolist()
y = df["dFF"].values

X0 = smiles_to_daylight(smiles)   # shape (N, 2048)

# -----------------------------
# BASELINE: full 2048-bit Daylight FP
# -----------------------------
r2s = []
baseline_txt = OUT_DIR / "r2_SVRlinear_baseline.txt"
with open(baseline_txt, "w") as f:
    f.write("# seed\tR2\n")
    for seed in SEEDS:
        X_tr, X_te, y_tr, y_te = train_test_split(
            X0, y, test_size=TEST_SIZE, random_state=int(seed)
        )
        model = SVR(kernel="linear")
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_te)
        r2 = r2_score(y_te, y_pred)
        r2s.append(r2)
        f.write(f"{int(seed)}\t{r2:.6f}\n")
baseline_mean = float(np.mean(r2s))
baseline_std  = float(np.std(r2s))
print(f"Daylight Baseline (2048 bits) -> Mean R2: {baseline_mean:.6f}, Std: {baseline_std:.6f}")

# -----------------------------
# RUN CASES 1–4
# -----------------------------
summary_rows = [{
    "Case": "baseline",
    "Orig_Dim": X0.shape[1],
    "Final_Dim": X0.shape[1],
    "R2_mean": baseline_mean,
    "R2_std": baseline_std,
    "Notes": "Full 2048-bit RDKit (Daylight-like) FP"
}]

for case in CASES:
    X_case, meta = transform_case_initial_only(X0, case)

    r2s = []
    out_txt = OUT_DIR / f"r2_SVRlinear_case{case}.txt"
    with open(out_txt, "w") as f:
        f.write("# seed\tR2\n")
        for seed in SEEDS:
            X_tr, X_te, y_tr, y_te = train_test_split(
                X_case, y, test_size=TEST_SIZE, random_state=int(seed)
            )
            model = SVR(kernel="linear")
            model.fit(X_tr, y_tr)
            y_pred = model.predict(X_te)
            r2 = r2_score(y_te, y_pred)
            r2s.append(r2)
            f.write(f"{int(seed)}\t{r2:.6f}\n")

    r2_mean = float(np.mean(r2s))
    r2_std  = float(np.std(r2s))

    summary_rows.append({
        "Case": case,
        "Orig_Dim": meta.get("orig_dim", X0.shape[1]),
        "Final_Dim": meta.get("final_dim", X_case.shape[1]),
        "R2_mean": r2_mean,
        "R2_std": r2_std,
        "Notes": {k:v for k,v in meta.items() if k not in ["orig_dim","final_dim"]}
    })

# -----------------------------
# SAVE SUMMARY
# -----------------------------
summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv(SUMMARY_CSV, index=False)

print("\n=== RDKit (Daylight-like) SVR (linear) Summary over 200 splits ===")
print(summary_df[["Case","Orig_Dim","Final_Dim","R2_mean","R2_std"]].to_string(index=False))
print(f"\nSaved summary CSV to: {SUMMARY_CSV}")


Daylight Baseline (2048 bits) -> Mean R2: 0.206734, Std: 0.408697

=== RDKit (Daylight-like) SVR (linear) Summary over 200 splits ===
    Case  Orig_Dim  Final_Dim  R2_mean   R2_std
baseline      2048       2048 0.206734 0.408697
       1      2048       1840 0.206734 0.408697
       2      2048       1202 0.208795 0.420486
  3_1024      2048       1024 0.165151 0.434407
   3_512      2048        512 0.074154 0.469219
  4_1024      2048        909 0.160846 0.442117
   4_512      2048        510 0.073289 0.469868

Saved summary CSV to: ml_runs_dimred_RDKFP/summary_results_RDKFP.csv


In [ ]:
# If in Colab, first:  !pip -q install rdkit openpyxl

import numpy as np
import pandas as pd
from pathlib import Path

from rdkit import Chem
from rdkit import DataStructs

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import r2_score

# -----------------------------
# CONFIG
# -----------------------------
N_BITS = 2048                 # Daylight-like FP size
MIN_PATH = 1
MAX_PATH = 7
BRANCHED_PATHS = True

N_SPLITS = 200
TEST_SIZE = 0.20
SEEDS = np.arange(N_SPLITS)   # 0..199

# Case-2 thresholds for your ~63-molecule dataset
RARE_MIN = 3
UBIQ_GT = 60

# Cases:
# 1: drop never-seen bits
# 2: drop never-seen, rare (<3), ubiquitous (>60)
# 3_k: fold 2048 -> k by modulo (no dropping)
# 4_k: fold, then Case-2 dropping on folded space
CASES = ["1", "2", "3_1024", "3_512", "4_1024", "4_512"]

INPUT_XLSX = "HOMO-LUMO-energies.xlsx"   # must have columns: Smiles, dFF
OUT_DIR = Path("ml_runs_dimred_RDKFP")
OUT_DIR.mkdir(parents=True, exist_ok=True)
SUMMARY_CSV = OUT_DIR / "summary_results_RDKFP.csv"

# -----------------------------
# UTILITIES
# -----------------------------
def smiles_to_daylight(smiles_list, fpSize=N_BITS, minPath=MIN_PATH, maxPath=MAX_PATH, branchedPaths=BRANCHED_PATHS):
    """Classic RDKit Daylight-like (path) fingerprint -> numpy array (N, fpSize)."""
    mols = [Chem.MolFromSmiles(s) for s in smiles_list]
    fps = [Chem.RDKFingerprint(m, fpSize=fpSize, minPath=minPath, maxPath=maxPath,
                               branchedPaths=branchedPaths) if m is not None else None
           for m in mols]
    X = np.zeros((len(fps), fpSize), dtype=np.uint8)
    for i, fp in enumerate(fps):
        if fp is None:
            continue
        DataStructs.ConvertToNumpyArray(fp, X[i])
    return X

def bit_presence(X):
    return (X > 0).astype(np.uint8)

def compute_freq_initial_only(X_init):
    # count in how many molecules each bit is present
    return bit_presence(X_init).sum(axis=0)

def select_cols_by_freq(freq, rule):
    if rule == "case1":        # drop never-seen
        return (freq > 0)
    elif rule == "case2":      # drop never-seen, rare (<3), ubiquitous (>60)
        return (freq > 0) & (freq >= RARE_MIN) & (freq <= UBIQ_GT)
    else:
        raise ValueError("rule must be 'case1' or 'case2'")

def apply_mask(X, mask):
    return X[:, mask]

def build_fold_map(orig_dim, new_dim):
    # modulo hashing: original column j -> j % new_dim
    return np.array([j % new_dim for j in range(orig_dim)], dtype=int)

def fold_binary_OR(X, idx_map):
    """Fold columns into new_dim buckets using OR aggregation."""
    new_dim = int(idx_map.max()) + 1
    out = np.zeros((X.shape[0], new_dim), dtype=np.uint8)
    for tgt in range(new_dim):
        cols = (idx_map == tgt)
        if np.any(cols):
            out[:, tgt] = np.max(X[:, cols], axis=1)
    return out

def transform_case_initial_only(X_init, case):
    """
    Case '1': drop never-seen bits
    Case '2': drop never-seen, rare (<3), ubiquitous (>60)
    Case '3_k': fold to k (no dropping)
    Case '4_k': fold to k, then Case-2 dropping on folded space
    """
    meta = {"orig_dim": X_init.shape[1]}
    if case in ("1", "2"):
        freq = compute_freq_initial_only(X_init)
        rule = "case1" if case == "1" else "case2"
        mask = select_cols_by_freq(freq, rule)
        X_out = apply_mask(X_init, mask)
        meta.update({
            "rule": rule,
            "final_dim": int(mask.sum()),
            "dropped": int((~mask).sum()),
        })
        return X_out, meta

    elif case.startswith("3_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(X_init.shape[1], new_dim)
        X_out = fold_binary_OR(X_init, idx_map)
        meta.update({
            "folded_to": new_dim,
            "final_dim": new_dim,
            "dropped": 0,
        })
        return X_out, meta

    elif case.startswith("4_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(X_init.shape[1], new_dim)
        X_fold = fold_binary_OR(X_init, idx_map)
        freq_fold = compute_freq_initial_only(X_fold)
        mask = select_cols_by_freq(freq_fold, "case2")
        X_out = apply_mask(X_fold, mask)
        meta.update({
            "folded_to": new_dim,
            "rule": "case2_after_fold",
            "final_dim": int(mask.sum()),
            "dropped_after_fold": int((~mask).sum()),
        })
        return X_out, meta

    else:
        raise ValueError(f"Unknown case: {case}")

# -----------------------------
# LOAD DATA & BUILD FEATURES
# -----------------------------
df = pd.read_excel(INPUT_XLSX)
smiles = df["Smiles"].astype(str).tolist()
y = df["dFF"].values

X0 = smiles_to_daylight(smiles)   # shape (N, 2048)

# -----------------------------
# BASELINE: full 2048-bit Daylight FP
# -----------------------------
r2s = []
baseline_txt = OUT_DIR / "r2_SVRsigmoid_baseline.txt"
with open(baseline_txt, "w") as f:
    f.write("# seed\tR2\n")
    for seed in SEEDS:
        X_tr, X_te, y_tr, y_te = train_test_split(
            X0, y, test_size=TEST_SIZE, random_state=int(seed)
        )
        model = SVR(kernel="sigmoid")
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_te)
        r2 = r2_score(y_te, y_pred)
        r2s.append(r2)
        f.write(f"{int(seed)}\t{r2:.6f}\n")
baseline_mean = float(np.mean(r2s))
baseline_std  = float(np.std(r2s))
print(f"Daylight Baseline (2048 bits) -> Mean R2: {baseline_mean:.6f}, Std: {baseline_std:.6f}")

# -----------------------------
# RUN CASES 1–4
# -----------------------------
summary_rows = [{
    "Case": "baseline",
    "Orig_Dim": X0.shape[1],
    "Final_Dim": X0.shape[1],
    "R2_mean": baseline_mean,
    "R2_std": baseline_std,
    "Notes": "Full 2048-bit RDKit (Daylight-like) FP"
}]

for case in CASES:
    X_case, meta = transform_case_initial_only(X0, case)

    r2s = []
    out_txt = OUT_DIR / f"r2_SVRsigmoid_case{case}.txt"
    with open(out_txt, "w") as f:
        f.write("# seed\tR2\n")
        for seed in SEEDS:
            X_tr, X_te, y_tr, y_te = train_test_split(
                X_case, y, test_size=TEST_SIZE, random_state=int(seed)
            )
            model = SVR(kernel="sigmoid")
            model.fit(X_tr, y_tr)
            y_pred = model.predict(X_te)
            r2 = r2_score(y_te, y_pred)
            r2s.append(r2)
            f.write(f"{int(seed)}\t{r2:.6f}\n")

    r2_mean = float(np.mean(r2s))
    r2_std  = float(np.std(r2s))

    summary_rows.append({
        "Case": case,
        "Orig_Dim": meta.get("orig_dim", X0.shape[1]),
        "Final_Dim": meta.get("final_dim", X_case.shape[1]),
        "R2_mean": r2_mean,
        "R2_std": r2_std,
        "Notes": {k:v for k,v in meta.items() if k not in ["orig_dim","final_dim"]}
    })

# -----------------------------
# SAVE SUMMARY
# -----------------------------
summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv(SUMMARY_CSV, index=False)

print("\n=== RDKit (Daylight-like) SVR (sigmoid) Summary over 200 splits ===")
print(summary_df[["Case","Orig_Dim","Final_Dim","R2_mean","R2_std"]].to_string(index=False))
print(f"\nSaved summary CSV to: {SUMMARY_CSV}")


Daylight Baseline (2048 bits) -> Mean R2: 0.112590, Std: 0.267176

=== RDKit (Daylight-like) SVR (sigmoid) Summary over 200 splits ===
    Case  Orig_Dim  Final_Dim  R2_mean   R2_std
baseline      2048       2048 0.112590 0.267176
       1      2048       1840 0.110392 0.268001
       2      2048       1202 0.107049 0.262094
  3_1024      2048       1024 0.073416 0.258831
   3_512      2048        512 0.028572 0.264219
  4_1024      2048        909 0.068406 0.259214
   4_512      2048        510 0.028306 0.264765

Saved summary CSV to: ml_runs_dimred_RDKFP/summary_results_RDKFP.csv


# Avalon

In [ ]:
# If in Colab, first:  !pip -q install rdkit-pypi openpyxl

import numpy as np
import pandas as pd
from pathlib import Path

from rdkit import Chem
from rdkit.Avalon import pyAvalonTools
from rdkit import DataStructs

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import r2_score

# -----------------------------
# CONFIG
# -----------------------------
N_BITS = 1024                  # Avalon default size
N_SPLITS = 200
TEST_SIZE = 0.20
SEEDS = np.arange(N_SPLITS)

# For Case 2 thresholds
RARE_MIN = 3
UBIQ_GT = 60

CASES = ["1", "2", "3_512", "3_256", "4_512", "4_256"]

INPUT_XLSX = "HOMO-LUMO-energies.xlsx"   # must have Smiles, dFF
OUT_DIR = Path("ml_runs_dimred_Avalon")
OUT_DIR.mkdir(parents=True, exist_ok=True)
SUMMARY_CSV = OUT_DIR / "summary_results_Avalon.csv"

# -----------------------------
# UTILITIES
# -----------------------------
def smiles_to_avalon(smiles_list, nBits=N_BITS):
    mols = [Chem.MolFromSmiles(s) for s in smiles_list]
    fps = [pyAvalonTools.GetAvalonFP(m, nBits) if m is not None else None for m in mols]
    X = np.zeros((len(fps), nBits), dtype=np.uint8)
    for i, fp in enumerate(fps):
        if fp is None:
            continue
        DataStructs.ConvertToNumpyArray(fp, X[i])
    return X

def bit_presence(X): return (X > 0).astype(np.uint8)
def compute_freq_initial_only(X_init): return bit_presence(X_init).sum(axis=0)

def select_cols_by_freq(freq, rule):
    if rule == "case1":
        return (freq > 0)
    elif rule == "case2":
        return (freq > 0) & (freq >= RARE_MIN) & (freq <= UBIQ_GT)
    else:
        raise ValueError("rule must be 'case1' or 'case2'")

def apply_mask(X, mask): return X[:, mask]
def build_fold_map(orig_dim, new_dim): return np.array([j % new_dim for j in range(orig_dim)], dtype=int)

def fold_binary_OR(X, idx_map):
    new_dim = idx_map.max() + 1
    out = np.zeros((X.shape[0], new_dim), dtype=np.uint8)
    for tgt in range(new_dim):
        cols = (idx_map == tgt)
        if np.any(cols):
            out[:, tgt] = np.max(X[:, cols], axis=1)
    return out

def transform_case_initial_only(X_init, case):
    meta = {"orig_dim": X_init.shape[1]}
    if case in ("1", "2"):
        freq = compute_freq_initial_only(X_init)
        rule = "case1" if case=="1" else "case2"
        mask = select_cols_by_freq(freq, rule)
        X_out = apply_mask(X_init, mask)
        meta.update({"rule": rule, "final_dim": int(mask.sum()), "dropped": int((~mask).sum())})
        return X_out, meta
    elif case.startswith("3_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(X_init.shape[1], new_dim)
        X_out = fold_binary_OR(X_init, idx_map)
        meta.update({"folded_to": new_dim, "final_dim": new_dim, "dropped": 0})
        return X_out, meta
    elif case.startswith("4_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(X_init.shape[1], new_dim)
        X_fold = fold_binary_OR(X_init, idx_map)
        freq_fold = compute_freq_initial_only(X_fold)
        mask = select_cols_by_freq(freq_fold, "case2")
        X_out = apply_mask(X_fold, mask)
        meta.update({"folded_to": new_dim, "rule": "case2_after_fold",
                     "final_dim": int(mask.sum()), "dropped_after_fold": int((~mask).sum())})
        return X_out, meta
    else:
        raise ValueError(f"Unknown case: {case}")

# -----------------------------
# LOAD DATA
# -----------------------------
df = pd.read_excel(INPUT_XLSX)
smiles = df["Smiles"].astype(str).tolist()
y = df["dFF"].values

X0 = smiles_to_avalon(smiles)   # shape (N, 1024)

# -----------------------------
# BASELINE (no reduction)
# -----------------------------
r2s = []
baseline_txt = OUT_DIR / "r2_SVRrbf_baseline.txt"
with open(baseline_txt, "w") as f:
    f.write("# seed\tR2\n")
    for seed in SEEDS:
        X_tr, X_te, y_tr, y_te = train_test_split(X0, y, test_size=TEST_SIZE, random_state=int(seed))
        model = SVR(kernel="rbf")
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_te)
        r2 = r2_score(y_te, y_pred)
        r2s.append(r2)
        f.write(f"{int(seed)}\t{r2:.6f}\n")
baseline_mean, baseline_std = float(np.mean(r2s)), float(np.std(r2s))
print(f"Avalon Baseline (1024 bits) -> Mean R2: {baseline_mean:.6f}, Std: {baseline_std:.6f}")

# -----------------------------
# RUN CASES 1–4
# -----------------------------
summary_rows = [{
    "Case": "baseline",
    "Orig_Dim": X0.shape[1],
    "Final_Dim": X0.shape[1],
    "R2_mean": baseline_mean,
    "R2_std": baseline_std,
    "Notes": "Full 1024-bit Avalon FP"
}]

for case in CASES:
    X_case, meta = transform_case_initial_only(X0, case)
    r2s = []
    out_txt = OUT_DIR / f"r2_SVRrbf_case{case}.txt"
    with open(out_txt, "w") as f:
        f.write("# seed\tR2\n")
        for seed in SEEDS:
            X_tr, X_te, y_tr, y_te = train_test_split(X_case, y, test_size=TEST_SIZE, random_state=int(seed))
            model = SVR(kernel="rbf")
            model.fit(X_tr, y_tr)
            y_pred = model.predict(X_te)
            r2 = r2_score(y_te, y_pred)
            r2s.append(r2)
            f.write(f"{int(seed)}\t{r2:.6f}\n")
    summary_rows.append({
        "Case": case,
        "Orig_Dim": meta.get("orig_dim", X0.shape[1]),
        "Final_Dim": meta.get("final_dim", X_case.shape[1]),
        "R2_mean": float(np.mean(r2s)),
        "R2_std": float(np.std(r2s)),
        "Notes": {k:v for k,v in meta.items() if k not in ["orig_dim","final_dim"]}
    })

# -----------------------------
# SAVE SUMMARY
# -----------------------------
summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv(SUMMARY_CSV, index=False)

print("\n=== Avalon SVR (RBF) Summary over 200 splits ===")
print(summary_df[["Case","Orig_Dim","Final_Dim","R2_mean","R2_std"]].to_string(index=False))
print(f"\nSaved summary CSV to: {SUMMARY_CSV}")


Avalon Baseline (1024 bits) -> Mean R2: 0.114430, Std: 0.252629

=== Avalon SVR (RBF) Summary over 200 splits ===
    Case  Orig_Dim  Final_Dim  R2_mean   R2_std
baseline      1024       1024 0.114430 0.252629
       1      1024        581 0.111698 0.253737
       2      1024        351 0.139344 0.247125
   3_512      1024        512 0.127676 0.251436
   3_256      1024        256 0.138115 0.241004
   4_512      1024        303 0.145358 0.249111
   4_256      1024        221 0.147480 0.239716

Saved summary CSV to: ml_runs_dimred_Avalon/summary_results_Avalon.csv


In [ ]:
# If in Colab, first:  !pip -q install rdkit-pypi openpyxl

import numpy as np
import pandas as pd
from pathlib import Path

from rdkit import Chem
from rdkit.Avalon import pyAvalonTools
from rdkit import DataStructs

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import r2_score

# -----------------------------
# CONFIG
# -----------------------------
N_BITS = 1024                  # Avalon default size
N_SPLITS = 200
TEST_SIZE = 0.20
SEEDS = np.arange(N_SPLITS)

# For Case 2 thresholds
RARE_MIN = 3
UBIQ_GT = 60

CASES = ["1", "2", "3_512", "3_256", "4_512", "4_256"]

INPUT_XLSX = "HOMO-LUMO-energies.xlsx"   # must have Smiles, dFF
OUT_DIR = Path("ml_runs_dimred_Avalon")
OUT_DIR.mkdir(parents=True, exist_ok=True)
SUMMARY_CSV = OUT_DIR / "summary_results_Avalon.csv"

# -----------------------------
# UTILITIES
# -----------------------------
def smiles_to_avalon(smiles_list, nBits=N_BITS):
    mols = [Chem.MolFromSmiles(s) for s in smiles_list]
    fps = [pyAvalonTools.GetAvalonFP(m, nBits) if m is not None else None for m in mols]
    X = np.zeros((len(fps), nBits), dtype=np.uint8)
    for i, fp in enumerate(fps):
        if fp is None:
            continue
        DataStructs.ConvertToNumpyArray(fp, X[i])
    return X

def bit_presence(X): return (X > 0).astype(np.uint8)
def compute_freq_initial_only(X_init): return bit_presence(X_init).sum(axis=0)

def select_cols_by_freq(freq, rule):
    if rule == "case1":
        return (freq > 0)
    elif rule == "case2":
        return (freq > 0) & (freq >= RARE_MIN) & (freq <= UBIQ_GT)
    else:
        raise ValueError("rule must be 'case1' or 'case2'")

def apply_mask(X, mask): return X[:, mask]
def build_fold_map(orig_dim, new_dim): return np.array([j % new_dim for j in range(orig_dim)], dtype=int)

def fold_binary_OR(X, idx_map):
    new_dim = idx_map.max() + 1
    out = np.zeros((X.shape[0], new_dim), dtype=np.uint8)
    for tgt in range(new_dim):
        cols = (idx_map == tgt)
        if np.any(cols):
            out[:, tgt] = np.max(X[:, cols], axis=1)
    return out

def transform_case_initial_only(X_init, case):
    meta = {"orig_dim": X_init.shape[1]}
    if case in ("1", "2"):
        freq = compute_freq_initial_only(X_init)
        rule = "case1" if case=="1" else "case2"
        mask = select_cols_by_freq(freq, rule)
        X_out = apply_mask(X_init, mask)
        meta.update({"rule": rule, "final_dim": int(mask.sum()), "dropped": int((~mask).sum())})
        return X_out, meta
    elif case.startswith("3_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(X_init.shape[1], new_dim)
        X_out = fold_binary_OR(X_init, idx_map)
        meta.update({"folded_to": new_dim, "final_dim": new_dim, "dropped": 0})
        return X_out, meta
    elif case.startswith("4_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(X_init.shape[1], new_dim)
        X_fold = fold_binary_OR(X_init, idx_map)
        freq_fold = compute_freq_initial_only(X_fold)
        mask = select_cols_by_freq(freq_fold, "case2")
        X_out = apply_mask(X_fold, mask)
        meta.update({"folded_to": new_dim, "rule": "case2_after_fold",
                     "final_dim": int(mask.sum()), "dropped_after_fold": int((~mask).sum())})
        return X_out, meta
    else:
        raise ValueError(f"Unknown case: {case}")

# -----------------------------
# LOAD DATA
# -----------------------------
df = pd.read_excel(INPUT_XLSX)
smiles = df["Smiles"].astype(str).tolist()
y = df["dFF"].values

X0 = smiles_to_avalon(smiles)   # shape (N, 1024)

# -----------------------------
# BASELINE (no reduction)
# -----------------------------
r2s = []
baseline_txt = OUT_DIR / "r2_SVRlinear_baseline.txt"
with open(baseline_txt, "w") as f:
    f.write("# seed\tR2\n")
    for seed in SEEDS:
        X_tr, X_te, y_tr, y_te = train_test_split(X0, y, test_size=TEST_SIZE, random_state=int(seed))
        model = SVR(kernel="linear")
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_te)
        r2 = r2_score(y_te, y_pred)
        r2s.append(r2)
        f.write(f"{int(seed)}\t{r2:.6f}\n")
baseline_mean, baseline_std = float(np.mean(r2s)), float(np.std(r2s))
print(f"Avalon Baseline (1024 bits) -> Mean R2: {baseline_mean:.6f}, Std: {baseline_std:.6f}")

# -----------------------------
# RUN CASES 1–4
# -----------------------------
summary_rows = [{
    "Case": "baseline",
    "Orig_Dim": X0.shape[1],
    "Final_Dim": X0.shape[1],
    "R2_mean": baseline_mean,
    "R2_std": baseline_std,
    "Notes": "Full 1024-bit Avalon FP"
}]

for case in CASES:
    X_case, meta = transform_case_initial_only(X0, case)
    r2s = []
    out_txt = OUT_DIR / f"r2_SVRlinear_case{case}.txt"
    with open(out_txt, "w") as f:
        f.write("# seed\tR2\n")
        for seed in SEEDS:
            X_tr, X_te, y_tr, y_te = train_test_split(X_case, y, test_size=TEST_SIZE, random_state=int(seed))
            model = SVR(kernel="linear")
            model.fit(X_tr, y_tr)
            y_pred = model.predict(X_te)
            r2 = r2_score(y_te, y_pred)
            r2s.append(r2)
            f.write(f"{int(seed)}\t{r2:.6f}\n")
    summary_rows.append({
        "Case": case,
        "Orig_Dim": meta.get("orig_dim", X0.shape[1]),
        "Final_Dim": meta.get("final_dim", X_case.shape[1]),
        "R2_mean": float(np.mean(r2s)),
        "R2_std": float(np.std(r2s)),
        "Notes": {k:v for k,v in meta.items() if k not in ["orig_dim","final_dim"]}
    })

# -----------------------------
# SAVE SUMMARY
# -----------------------------
summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv(SUMMARY_CSV, index=False)

print("\n=== Avalon SVR (linear) Summary over 200 splits ===")
print(summary_df[["Case","Orig_Dim","Final_Dim","R2_mean","R2_std"]].to_string(index=False))
print(f"\nSaved summary CSV to: {SUMMARY_CSV}")


Avalon Baseline (1024 bits) -> Mean R2: 0.189788, Std: 0.446367

=== Avalon SVR (linear) Summary over 200 splits ===
    Case  Orig_Dim  Final_Dim   R2_mean   R2_std
baseline      1024       1024  0.189788 0.446367
       1      1024        581  0.189788 0.446367
       2      1024        351  0.166303 0.476478
   3_512      1024        512  0.165034 0.419184
   3_256      1024        256 -0.039079 0.499682
   4_512      1024        303  0.131953 0.430210
   4_256      1024        221 -0.076858 0.512474

Saved summary CSV to: ml_runs_dimred_Avalon/summary_results_Avalon.csv


In [ ]:
# If in Colab, first:  !pip -q install rdkit-pypi openpyxl

import numpy as np
import pandas as pd
from pathlib import Path

from rdkit import Chem
from rdkit.Avalon import pyAvalonTools
from rdkit import DataStructs

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import r2_score

# -----------------------------
# CONFIG
# -----------------------------
N_BITS = 1024                  # Avalon default size
N_SPLITS = 200
TEST_SIZE = 0.20
SEEDS = np.arange(N_SPLITS)

# For Case 2 thresholds
RARE_MIN = 3
UBIQ_GT = 60

CASES = ["1", "2", "3_512", "3_256", "4_512", "4_256"]

INPUT_XLSX = "HOMO-LUMO-energies.xlsx"   # must have Smiles, dFF
OUT_DIR = Path("ml_runs_dimred_Avalon")
OUT_DIR.mkdir(parents=True, exist_ok=True)
SUMMARY_CSV = OUT_DIR / "summary_results_Avalon.csv"

# -----------------------------
# UTILITIES
# -----------------------------
def smiles_to_avalon(smiles_list, nBits=N_BITS):
    mols = [Chem.MolFromSmiles(s) for s in smiles_list]
    fps = [pyAvalonTools.GetAvalonFP(m, nBits) if m is not None else None for m in mols]
    X = np.zeros((len(fps), nBits), dtype=np.uint8)
    for i, fp in enumerate(fps):
        if fp is None:
            continue
        DataStructs.ConvertToNumpyArray(fp, X[i])
    return X

def bit_presence(X): return (X > 0).astype(np.uint8)
def compute_freq_initial_only(X_init): return bit_presence(X_init).sum(axis=0)

def select_cols_by_freq(freq, rule):
    if rule == "case1":
        return (freq > 0)
    elif rule == "case2":
        return (freq > 0) & (freq >= RARE_MIN) & (freq <= UBIQ_GT)
    else:
        raise ValueError("rule must be 'case1' or 'case2'")

def apply_mask(X, mask): return X[:, mask]
def build_fold_map(orig_dim, new_dim): return np.array([j % new_dim for j in range(orig_dim)], dtype=int)

def fold_binary_OR(X, idx_map):
    new_dim = idx_map.max() + 1
    out = np.zeros((X.shape[0], new_dim), dtype=np.uint8)
    for tgt in range(new_dim):
        cols = (idx_map == tgt)
        if np.any(cols):
            out[:, tgt] = np.max(X[:, cols], axis=1)
    return out

def transform_case_initial_only(X_init, case):
    meta = {"orig_dim": X_init.shape[1]}
    if case in ("1", "2"):
        freq = compute_freq_initial_only(X_init)
        rule = "case1" if case=="1" else "case2"
        mask = select_cols_by_freq(freq, rule)
        X_out = apply_mask(X_init, mask)
        meta.update({"rule": rule, "final_dim": int(mask.sum()), "dropped": int((~mask).sum())})
        return X_out, meta
    elif case.startswith("3_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(X_init.shape[1], new_dim)
        X_out = fold_binary_OR(X_init, idx_map)
        meta.update({"folded_to": new_dim, "final_dim": new_dim, "dropped": 0})
        return X_out, meta
    elif case.startswith("4_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(X_init.shape[1], new_dim)
        X_fold = fold_binary_OR(X_init, idx_map)
        freq_fold = compute_freq_initial_only(X_fold)
        mask = select_cols_by_freq(freq_fold, "case2")
        X_out = apply_mask(X_fold, mask)
        meta.update({"folded_to": new_dim, "rule": "case2_after_fold",
                     "final_dim": int(mask.sum()), "dropped_after_fold": int((~mask).sum())})
        return X_out, meta
    else:
        raise ValueError(f"Unknown case: {case}")

# -----------------------------
# LOAD DATA
# -----------------------------
df = pd.read_excel(INPUT_XLSX)
smiles = df["Smiles"].astype(str).tolist()
y = df["dFF"].values

X0 = smiles_to_avalon(smiles)   # shape (N, 1024)

# -----------------------------
# BASELINE (no reduction)
# -----------------------------
r2s = []
baseline_txt = OUT_DIR / "r2_SVRsigmoid_baseline.txt"
with open(baseline_txt, "w") as f:
    f.write("# seed\tR2\n")
    for seed in SEEDS:
        X_tr, X_te, y_tr, y_te = train_test_split(X0, y, test_size=TEST_SIZE, random_state=int(seed))
        model = SVR(kernel="sigmoid")
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_te)
        r2 = r2_score(y_te, y_pred)
        r2s.append(r2)
        f.write(f"{int(seed)}\t{r2:.6f}\n")
baseline_mean, baseline_std = float(np.mean(r2s)), float(np.std(r2s))
print(f"Avalon Baseline (1024 bits) -> Mean R2: {baseline_mean:.6f}, Std: {baseline_std:.6f}")

# -----------------------------
# RUN CASES 1–4
# -----------------------------
summary_rows = [{
    "Case": "baseline",
    "Orig_Dim": X0.shape[1],
    "Final_Dim": X0.shape[1],
    "R2_mean": baseline_mean,
    "R2_std": baseline_std,
    "Notes": "Full 1024-bit Avalon FP"
}]

for case in CASES:
    X_case, meta = transform_case_initial_only(X0, case)
    r2s = []
    out_txt = OUT_DIR / f"r2_SVRsigmoid_case{case}.txt"
    with open(out_txt, "w") as f:
        f.write("# seed\tR2\n")
        for seed in SEEDS:
            X_tr, X_te, y_tr, y_te = train_test_split(X_case, y, test_size=TEST_SIZE, random_state=int(seed))
            model = SVR(kernel="sigmoid")
            model.fit(X_tr, y_tr)
            y_pred = model.predict(X_te)
            r2 = r2_score(y_te, y_pred)
            r2s.append(r2)
            f.write(f"{int(seed)}\t{r2:.6f}\n")
    summary_rows.append({
        "Case": case,
        "Orig_Dim": meta.get("orig_dim", X0.shape[1]),
        "Final_Dim": meta.get("final_dim", X_case.shape[1]),
        "R2_mean": float(np.mean(r2s)),
        "R2_std": float(np.std(r2s)),
        "Notes": {k:v for k,v in meta.items() if k not in ["orig_dim","final_dim"]}
    })

# -----------------------------
# SAVE SUMMARY
# -----------------------------
summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv(SUMMARY_CSV, index=False)

print("\n=== Avalon SVR (sigmoid) Summary over 200 splits ===")
print(summary_df[["Case","Orig_Dim","Final_Dim","R2_mean","R2_std"]].to_string(index=False))
print(f"\nSaved summary CSV to: {SUMMARY_CSV}")


Avalon Baseline (1024 bits) -> Mean R2: 0.099085, Std: 0.292702

=== Avalon SVR (sigmoid) Summary over 200 splits ===
    Case  Orig_Dim  Final_Dim   R2_mean   R2_std
baseline      1024       1024  0.099085 0.292702
       1      1024        581  0.083899 0.302781
       2      1024        351  0.023829 0.343208
   3_512      1024        512 -0.002558 0.358618
   3_256      1024        256 -0.062751 0.308133
   4_512      1024        303 -0.033459 0.366015
   4_256      1024        221 -0.085137 0.316012

Saved summary CSV to: ml_runs_dimred_Avalon/summary_results_Avalon.csv


# AtomPairs

In [ ]:
# If in Colab (run once per runtime):
# !pip -q install rdkit openpyxl

import numpy as np
import pandas as pd
from pathlib import Path

from rdkit import Chem, DataStructs
from rdkit.Chem import rdFingerprintGenerator as rfg  # AtomPairGenerator

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import r2_score

# -----------------------------
# CONFIG
# -----------------------------
N_BITS = 2048
N_SPLITS = 200
TEST_SIZE = 0.20
SEEDS = np.arange(N_SPLITS)  # 0..199

# For Case 2 thresholds (your ~63-molecule set)
RARE_MIN = 3
UBIQ_GT = 60

# Cases:
# 1: drop never-seen bits
# 2: drop never-seen, rare (<3), ubiquitous (>60)
# 3_k: fold 2048 -> k by modulo (no dropping)
# 4_k: fold, then Case-2 dropping on folded space
CASES = ["1", "2", "3_1024", "3_512", "4_1024", "4_512"]

INPUT_XLSX = "HOMO-LUMO-energies.xlsx"   # must have columns: Smiles, dFF
OUT_DIR = Path("ml_runs_dimred_AtomPairs_Gen")
OUT_DIR.mkdir(parents=True, exist_ok=True)
SUMMARY_CSV = OUT_DIR / "summary_results_AtomPairs_Gen.csv"

# -----------------------------
# UTILITIES
# -----------------------------
def smiles_to_atompairs(smiles_list, nBits=N_BITS, minDistance=1, maxDistance=30, includeChirality=False):
    """
    RDKit AtomPairGenerator -> numpy array (N, nBits).
    Distances are in bonds; defaults are typical for AtomPairs.
    """
    gen = rfg.GetAtomPairGenerator(
        fpSize=nBits,
        minDistance=minDistance,
        maxDistance=maxDistance,
        includeChirality=includeChirality
    )
    mols = [Chem.MolFromSmiles(s) for s in smiles_list]
    X = np.zeros((len(mols), nBits), dtype=np.uint8)
    for i, m in enumerate(mols):
        if m is None:
            continue
        fp = gen.GetFingerprint(m)                 # ExplicitBitVect
        DataStructs.ConvertToNumpyArray(fp, X[i])  # fills row in-place
    return X

def bit_presence(X): return (X > 0).astype(np.uint8)

def compute_freq_initial_only(X_init):
    # in how many molecules each bit is present
    return bit_presence(X_init).sum(axis=0)

def select_cols_by_freq(freq, rule):
    if rule == "case1":        # drop never-seen
        return (freq > 0)
    elif rule == "case2":      # drop never-seen, rare (<3), ubiquitous (>60)
        return (freq > 0) & (freq >= RARE_MIN) & (freq <= UBIQ_GT)
    else:
        raise ValueError("rule must be 'case1' or 'case2'")

def apply_mask(X, mask): return X[:, mask]

def build_fold_map(orig_dim, new_dim):
    # modulo hashing: original column j -> j % new_dim
    return np.array([j % new_dim for j in range(orig_dim)], dtype=int)

def fold_binary_OR(X, idx_map):
    """Fold columns into new_dim buckets using OR aggregation (binary fingerprints)."""
    new_dim = idx_map.max() + 1
    out = np.zeros((X.shape[0], new_dim), dtype=np.uint8)
    for tgt in range(new_dim):
        cols = (idx_map == tgt)
        if np.any(cols):
            out[:, tgt] = np.max(X[:, cols], axis=1)
    return out

def transform_case_initial_only(X_init, case):
    """
    Case '1': drop never-seen bits
    Case '2': drop never-seen, rare (<3), ubiquitous (>60)
    Case '3_k': fold to k (no dropping)
    Case '4_k': fold to k, then Case-2 dropping on folded space
    """
    meta = {"orig_dim": X_init.shape[1]}
    if case in ("1", "2"):
        freq = compute_freq_initial_only(X_init)
        rule = "case1" if case == "1" else "case2"
        mask = select_cols_by_freq(freq, rule)
        X_out = apply_mask(X_init, mask)
        meta.update({"rule": rule, "final_dim": int(mask.sum()), "dropped": int((~mask).sum())})
        return X_out, meta

    elif case.startswith("3_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(X_init.shape[1], new_dim)
        X_out = fold_binary_OR(X_init, idx_map)
        meta.update({"folded_to": new_dim, "final_dim": new_dim, "dropped": 0})
        return X_out, meta

    elif case.startswith("4_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(X_init.shape[1], new_dim)
        X_fold = fold_binary_OR(X_init, idx_map)
        freq_fold = compute_freq_initial_only(X_fold)
        mask = select_cols_by_freq(freq_fold, "case2")
        X_out = apply_mask(X_fold, mask)
        meta.update({"folded_to": new_dim, "rule": "case2_after_fold",
                     "final_dim": int(mask.sum()), "dropped_after_fold": int((~mask).sum())})
        return X_out, meta

    else:
        raise ValueError(f"Unknown case: {case}")

# -----------------------------
# LOAD DATA
# -----------------------------
df = pd.read_excel(INPUT_XLSX)
smiles = df["Smiles"].astype(str).tolist()
y = df["dFF"].values

X0 = smiles_to_atompairs(smiles)  # (N, 2048)

# -----------------------------
# BASELINE (no reduction)
# -----------------------------
r2s = []
baseline_txt = OUT_DIR / "r2_SVRrbf_baseline.txt"
with open(baseline_txt, "w") as f:
    f.write("# seed\tR2\n")
    for seed in SEEDS:
        X_tr, X_te, y_tr, y_te = train_test_split(X0, y, test_size=TEST_SIZE, random_state=int(seed))
        model = SVR(kernel="rbf")
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_te)
        r2 = r2_score(y_te, y_pred)
        r2s.append(r2)
        f.write(f"{int(seed)}\t{r2:.6f}\n")
baseline_mean, baseline_std = float(np.mean(r2s)), float(np.std(r2s))
print(f"AtomPairs (Generator) Baseline (2048 bits) -> Mean R2: {baseline_mean:.6f}, Std: {baseline_std:.6f}")

# -----------------------------
# RUN CASES 1–4
# -----------------------------
summary_rows = [{
    "Case": "baseline",
    "Orig_Dim": X0.shape[1],
    "Final_Dim": X0.shape[1],
    "R2_mean": baseline_mean,
    "R2_std": baseline_std,
    "Notes": "Full 2048-bit AtomPairs (AtomPairGenerator)"
}]

for case in CASES:
    X_case, meta = transform_case_initial_only(X0, case)
    r2s = []
    out_txt = OUT_DIR / f"r2_SVRrbf_case{case}.txt"
    with open(out_txt, "w") as f:
        f.write("# seed\tR2\n")
        for seed in SEEDS:
            X_tr, X_te, y_tr, y_te = train_test_split(X_case, y, test_size=TEST_SIZE, random_state=int(seed))
            model = SVR(kernel="rbf")
            model.fit(X_tr, y_tr)
            y_pred = model.predict(X_te)
            r2 = r2_score(y_te, y_pred)
            r2s.append(r2)
            f.write(f"{int(seed)}\t{r2:.6f}\n")
    summary_rows.append({
        "Case": case,
        "Orig_Dim": meta.get("orig_dim", X0.shape[1]),
        "Final_Dim": meta.get("final_dim", X_case.shape[1]),
        "R2_mean": float(np.mean(r2s)),
        "R2_std": float(np.std(r2s)),
        "Notes": {k:v for k,v in meta.items() if k not in ["orig_dim","final_dim"]}
    })

# -----------------------------
# SAVE SUMMARY
# -----------------------------
summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv(SUMMARY_CSV, index=False)

print("\n=== AtomPairs (Generator) SVR (RBF) Summary over 200 splits ===")
print(summary_df[["Case","Orig_Dim","Final_Dim","R2_mean","R2_std"]].to_string(index=False))
print(f"\nSaved summary CSV to: {SUMMARY_CSV}")


AtomPairs (Generator) Baseline (2048 bits) -> Mean R2: 0.129293, Std: 0.271291

=== AtomPairs (Generator) SVR (RBF) Summary over 200 splits ===
    Case  Orig_Dim  Final_Dim  R2_mean   R2_std
baseline      2048       2048 0.129293 0.271291
       1      2048        480 0.120355 0.272957
       2      2048        219 0.164116 0.265656
  3_1024      2048       1024 0.146616 0.270217
   3_512      2048        512 0.158734 0.267656
  4_1024      2048        210 0.174260 0.268691
   4_512      2048        184 0.168379 0.272046

Saved summary CSV to: ml_runs_dimred_AtomPairs_Gen/summary_results_AtomPairs_Gen.csv


In [ ]:
# If in Colab (run once per runtime):
# !pip -q install rdkit openpyxl

import numpy as np
import pandas as pd
from pathlib import Path

from rdkit import Chem, DataStructs
from rdkit.Chem import rdFingerprintGenerator as rfg  # AtomPairGenerator

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import r2_score

# -----------------------------
# CONFIG
# -----------------------------
N_BITS = 2048
N_SPLITS = 200
TEST_SIZE = 0.20
SEEDS = np.arange(N_SPLITS)  # 0..199

# For Case 2 thresholds (your ~63-molecule set)
RARE_MIN = 3
UBIQ_GT = 60

# Cases:
# 1: drop never-seen bits
# 2: drop never-seen, rare (<3), ubiquitous (>60)
# 3_k: fold 2048 -> k by modulo (no dropping)
# 4_k: fold, then Case-2 dropping on folded space
CASES = ["1", "2", "3_1024", "3_512", "4_1024", "4_512"]

INPUT_XLSX = "HOMO-LUMO-energies.xlsx"   # must have columns: Smiles, dFF
OUT_DIR = Path("ml_runs_dimred_AtomPairs_Gen")
OUT_DIR.mkdir(parents=True, exist_ok=True)
SUMMARY_CSV = OUT_DIR / "summary_results_AtomPairs_Gen.csv"

# -----------------------------
# UTILITIES
# -----------------------------
def smiles_to_atompairs(smiles_list, nBits=N_BITS, minDistance=1, maxDistance=30, includeChirality=False):
    """
    RDKit AtomPairGenerator -> numpy array (N, nBits).
    Distances are in bonds; defaults are typical for AtomPairs.
    """
    gen = rfg.GetAtomPairGenerator(
        fpSize=nBits,
        minDistance=minDistance,
        maxDistance=maxDistance,
        includeChirality=includeChirality
    )
    mols = [Chem.MolFromSmiles(s) for s in smiles_list]
    X = np.zeros((len(mols), nBits), dtype=np.uint8)
    for i, m in enumerate(mols):
        if m is None:
            continue
        fp = gen.GetFingerprint(m)                 # ExplicitBitVect
        DataStructs.ConvertToNumpyArray(fp, X[i])  # fills row in-place
    return X

def bit_presence(X): return (X > 0).astype(np.uint8)

def compute_freq_initial_only(X_init):
    # in how many molecules each bit is present
    return bit_presence(X_init).sum(axis=0)

def select_cols_by_freq(freq, rule):
    if rule == "case1":        # drop never-seen
        return (freq > 0)
    elif rule == "case2":      # drop never-seen, rare (<3), ubiquitous (>60)
        return (freq > 0) & (freq >= RARE_MIN) & (freq <= UBIQ_GT)
    else:
        raise ValueError("rule must be 'case1' or 'case2'")

def apply_mask(X, mask): return X[:, mask]

def build_fold_map(orig_dim, new_dim):
    # modulo hashing: original column j -> j % new_dim
    return np.array([j % new_dim for j in range(orig_dim)], dtype=int)

def fold_binary_OR(X, idx_map):
    """Fold columns into new_dim buckets using OR aggregation (binary fingerprints)."""
    new_dim = idx_map.max() + 1
    out = np.zeros((X.shape[0], new_dim), dtype=np.uint8)
    for tgt in range(new_dim):
        cols = (idx_map == tgt)
        if np.any(cols):
            out[:, tgt] = np.max(X[:, cols], axis=1)
    return out

def transform_case_initial_only(X_init, case):
    """
    Case '1': drop never-seen bits
    Case '2': drop never-seen, rare (<3), ubiquitous (>60)
    Case '3_k': fold to k (no dropping)
    Case '4_k': fold to k, then Case-2 dropping on folded space
    """
    meta = {"orig_dim": X_init.shape[1]}
    if case in ("1", "2"):
        freq = compute_freq_initial_only(X_init)
        rule = "case1" if case == "1" else "case2"
        mask = select_cols_by_freq(freq, rule)
        X_out = apply_mask(X_init, mask)
        meta.update({"rule": rule, "final_dim": int(mask.sum()), "dropped": int((~mask).sum())})
        return X_out, meta

    elif case.startswith("3_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(X_init.shape[1], new_dim)
        X_out = fold_binary_OR(X_init, idx_map)
        meta.update({"folded_to": new_dim, "final_dim": new_dim, "dropped": 0})
        return X_out, meta

    elif case.startswith("4_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(X_init.shape[1], new_dim)
        X_fold = fold_binary_OR(X_init, idx_map)
        freq_fold = compute_freq_initial_only(X_fold)
        mask = select_cols_by_freq(freq_fold, "case2")
        X_out = apply_mask(X_fold, mask)
        meta.update({"folded_to": new_dim, "rule": "case2_after_fold",
                     "final_dim": int(mask.sum()), "dropped_after_fold": int((~mask).sum())})
        return X_out, meta

    else:
        raise ValueError(f"Unknown case: {case}")

# -----------------------------
# LOAD DATA
# -----------------------------
df = pd.read_excel(INPUT_XLSX)
smiles = df["Smiles"].astype(str).tolist()
y = df["dFF"].values

X0 = smiles_to_atompairs(smiles)  # (N, 2048)

# -----------------------------
# BASELINE (no reduction)
# -----------------------------
r2s = []
baseline_txt = OUT_DIR / "r2_SVRlinear_baseline.txt"
with open(baseline_txt, "w") as f:
    f.write("# seed\tR2\n")
    for seed in SEEDS:
        X_tr, X_te, y_tr, y_te = train_test_split(X0, y, test_size=TEST_SIZE, random_state=int(seed))
        model = SVR(kernel="linear")
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_te)
        r2 = r2_score(y_te, y_pred)
        r2s.append(r2)
        f.write(f"{int(seed)}\t{r2:.6f}\n")
baseline_mean, baseline_std = float(np.mean(r2s)), float(np.std(r2s))
print(f"AtomPairs (Generator) Baseline (2048 bits) -> Mean R2: {baseline_mean:.6f}, Std: {baseline_std:.6f}")

# -----------------------------
# RUN CASES 1–4
# -----------------------------
summary_rows = [{
    "Case": "baseline",
    "Orig_Dim": X0.shape[1],
    "Final_Dim": X0.shape[1],
    "R2_mean": baseline_mean,
    "R2_std": baseline_std,
    "Notes": "Full 2048-bit AtomPairs (AtomPairGenerator)"
}]

for case in CASES:
    X_case, meta = transform_case_initial_only(X0, case)
    r2s = []
    out_txt = OUT_DIR / f"r2_SVRlinear_case{case}.txt"
    with open(out_txt, "w") as f:
        f.write("# seed\tR2\n")
        for seed in SEEDS:
            X_tr, X_te, y_tr, y_te = train_test_split(X_case, y, test_size=TEST_SIZE, random_state=int(seed))
            model = SVR(kernel="linear")
            model.fit(X_tr, y_tr)
            y_pred = model.predict(X_te)
            r2 = r2_score(y_te, y_pred)
            r2s.append(r2)
            f.write(f"{int(seed)}\t{r2:.6f}\n")
    summary_rows.append({
        "Case": case,
        "Orig_Dim": meta.get("orig_dim", X0.shape[1]),
        "Final_Dim": meta.get("final_dim", X_case.shape[1]),
        "R2_mean": float(np.mean(r2s)),
        "R2_std": float(np.std(r2s)),
        "Notes": {k:v for k,v in meta.items() if k not in ["orig_dim","final_dim"]}
    })

# -----------------------------
# SAVE SUMMARY
# -----------------------------
summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv(SUMMARY_CSV, index=False)

print("\n=== AtomPairs (Generator) SVR (linear) Summary over 200 splits ===")
print(summary_df[["Case","Orig_Dim","Final_Dim","R2_mean","R2_std"]].to_string(index=False))
print(f"\nSaved summary CSV to: {SUMMARY_CSV}")


AtomPairs (Generator) Baseline (2048 bits) -> Mean R2: 0.265278, Std: 0.413454

=== AtomPairs (Generator) SVR (linear) Summary over 200 splits ===
    Case  Orig_Dim  Final_Dim   R2_mean   R2_std
baseline      2048       2048  0.265278 0.413454
       1      2048        480  0.265278 0.413454
       2      2048        219  0.138232 0.459387
  3_1024      2048       1024  0.263502 0.421402
   3_512      2048        512  0.175075 0.482534
  4_1024      2048        210  0.153155 0.455106
   4_512      2048        184 -0.049713 0.590116

Saved summary CSV to: ml_runs_dimred_AtomPairs_Gen/summary_results_AtomPairs_Gen.csv


In [ ]:
# If in Colab (run once per runtime):
# !pip -q install rdkit openpyxl

import numpy as np
import pandas as pd
from pathlib import Path

from rdkit import Chem, DataStructs
from rdkit.Chem import rdFingerprintGenerator as rfg  # AtomPairGenerator

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import r2_score

# -----------------------------
# CONFIG
# -----------------------------
N_BITS = 2048
N_SPLITS = 200
TEST_SIZE = 0.20
SEEDS = np.arange(N_SPLITS)  # 0..199

# For Case 2 thresholds (your ~63-molecule set)
RARE_MIN = 3
UBIQ_GT = 60

# Cases:
# 1: drop never-seen bits
# 2: drop never-seen, rare (<3), ubiquitous (>60)
# 3_k: fold 2048 -> k by modulo (no dropping)
# 4_k: fold, then Case-2 dropping on folded space
CASES = ["1", "2", "3_1024", "3_512", "4_1024", "4_512"]

INPUT_XLSX = "HOMO-LUMO-energies.xlsx"   # must have columns: Smiles, dFF
OUT_DIR = Path("ml_runs_dimred_AtomPairs_Gen")
OUT_DIR.mkdir(parents=True, exist_ok=True)
SUMMARY_CSV = OUT_DIR / "summary_results_AtomPairs_Gen.csv"

# -----------------------------
# UTILITIES
# -----------------------------
def smiles_to_atompairs(smiles_list, nBits=N_BITS, minDistance=1, maxDistance=30, includeChirality=False):
    """
    RDKit AtomPairGenerator -> numpy array (N, nBits).
    Distances are in bonds; defaults are typical for AtomPairs.
    """
    gen = rfg.GetAtomPairGenerator(
        fpSize=nBits,
        minDistance=minDistance,
        maxDistance=maxDistance,
        includeChirality=includeChirality
    )
    mols = [Chem.MolFromSmiles(s) for s in smiles_list]
    X = np.zeros((len(mols), nBits), dtype=np.uint8)
    for i, m in enumerate(mols):
        if m is None:
            continue
        fp = gen.GetFingerprint(m)                 # ExplicitBitVect
        DataStructs.ConvertToNumpyArray(fp, X[i])  # fills row in-place
    return X

def bit_presence(X): return (X > 0).astype(np.uint8)

def compute_freq_initial_only(X_init):
    # in how many molecules each bit is present
    return bit_presence(X_init).sum(axis=0)

def select_cols_by_freq(freq, rule):
    if rule == "case1":        # drop never-seen
        return (freq > 0)
    elif rule == "case2":      # drop never-seen, rare (<3), ubiquitous (>60)
        return (freq > 0) & (freq >= RARE_MIN) & (freq <= UBIQ_GT)
    else:
        raise ValueError("rule must be 'case1' or 'case2'")

def apply_mask(X, mask): return X[:, mask]

def build_fold_map(orig_dim, new_dim):
    # modulo hashing: original column j -> j % new_dim
    return np.array([j % new_dim for j in range(orig_dim)], dtype=int)

def fold_binary_OR(X, idx_map):
    """Fold columns into new_dim buckets using OR aggregation (binary fingerprints)."""
    new_dim = idx_map.max() + 1
    out = np.zeros((X.shape[0], new_dim), dtype=np.uint8)
    for tgt in range(new_dim):
        cols = (idx_map == tgt)
        if np.any(cols):
            out[:, tgt] = np.max(X[:, cols], axis=1)
    return out

def transform_case_initial_only(X_init, case):
    """
    Case '1': drop never-seen bits
    Case '2': drop never-seen, rare (<3), ubiquitous (>60)
    Case '3_k': fold to k (no dropping)
    Case '4_k': fold to k, then Case-2 dropping on folded space
    """
    meta = {"orig_dim": X_init.shape[1]}
    if case in ("1", "2"):
        freq = compute_freq_initial_only(X_init)
        rule = "case1" if case == "1" else "case2"
        mask = select_cols_by_freq(freq, rule)
        X_out = apply_mask(X_init, mask)
        meta.update({"rule": rule, "final_dim": int(mask.sum()), "dropped": int((~mask).sum())})
        return X_out, meta

    elif case.startswith("3_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(X_init.shape[1], new_dim)
        X_out = fold_binary_OR(X_init, idx_map)
        meta.update({"folded_to": new_dim, "final_dim": new_dim, "dropped": 0})
        return X_out, meta

    elif case.startswith("4_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(X_init.shape[1], new_dim)
        X_fold = fold_binary_OR(X_init, idx_map)
        freq_fold = compute_freq_initial_only(X_fold)
        mask = select_cols_by_freq(freq_fold, "case2")
        X_out = apply_mask(X_fold, mask)
        meta.update({"folded_to": new_dim, "rule": "case2_after_fold",
                     "final_dim": int(mask.sum()), "dropped_after_fold": int((~mask).sum())})
        return X_out, meta

    else:
        raise ValueError(f"Unknown case: {case}")

# -----------------------------
# LOAD DATA
# -----------------------------
df = pd.read_excel(INPUT_XLSX)
smiles = df["Smiles"].astype(str).tolist()
y = df["dFF"].values

X0 = smiles_to_atompairs(smiles)  # (N, 2048)

# -----------------------------
# BASELINE (no reduction)
# -----------------------------
r2s = []
baseline_txt = OUT_DIR / "r2_SVRsigmoid_baseline.txt"
with open(baseline_txt, "w") as f:
    f.write("# seed\tR2\n")
    for seed in SEEDS:
        X_tr, X_te, y_tr, y_te = train_test_split(X0, y, test_size=TEST_SIZE, random_state=int(seed))
        model = SVR(kernel="sigmoid")
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_te)
        r2 = r2_score(y_te, y_pred)
        r2s.append(r2)
        f.write(f"{int(seed)}\t{r2:.6f}\n")
baseline_mean, baseline_std = float(np.mean(r2s)), float(np.std(r2s))
print(f"AtomPairs (Generator) Baseline (2048 bits) -> Mean R2: {baseline_mean:.6f}, Std: {baseline_std:.6f}")

# -----------------------------
# RUN CASES 1–4
# -----------------------------
summary_rows = [{
    "Case": "baseline",
    "Orig_Dim": X0.shape[1],
    "Final_Dim": X0.shape[1],
    "R2_mean": baseline_mean,
    "R2_std": baseline_std,
    "Notes": "Full 2048-bit AtomPairs (AtomPairGenerator)"
}]

for case in CASES:
    X_case, meta = transform_case_initial_only(X0, case)
    r2s = []
    out_txt = OUT_DIR / f"r2_SVRsigmoid_case{case}.txt"
    with open(out_txt, "w") as f:
        f.write("# seed\tR2\n")
        for seed in SEEDS:
            X_tr, X_te, y_tr, y_te = train_test_split(X_case, y, test_size=TEST_SIZE, random_state=int(seed))
            model = SVR(kernel="sigmoid")
            model.fit(X_tr, y_tr)
            y_pred = model.predict(X_te)
            r2 = r2_score(y_te, y_pred)
            r2s.append(r2)
            f.write(f"{int(seed)}\t{r2:.6f}\n")
    summary_rows.append({
        "Case": case,
        "Orig_Dim": meta.get("orig_dim", X0.shape[1]),
        "Final_Dim": meta.get("final_dim", X_case.shape[1]),
        "R2_mean": float(np.mean(r2s)),
        "R2_std": float(np.std(r2s)),
        "Notes": {k:v for k,v in meta.items() if k not in ["orig_dim","final_dim"]}
    })

# -----------------------------
# SAVE SUMMARY
# -----------------------------
summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv(SUMMARY_CSV, index=False)

print("\n=== AtomPairs (Generator) SVR (sigmoid) Summary over 200 splits ===")
print(summary_df[["Case","Orig_Dim","Final_Dim","R2_mean","R2_std"]].to_string(index=False))
print(f"\nSaved summary CSV to: {SUMMARY_CSV}")


AtomPairs (Generator) Baseline (2048 bits) -> Mean R2: 0.192114, Std: 0.254970

=== AtomPairs (Generator) SVR (sigmoid) Summary over 200 splits ===
    Case  Orig_Dim  Final_Dim   R2_mean   R2_std
baseline      2048       2048  0.192114 0.254970
       1      2048        480  0.178334 0.260065
       2      2048        219  0.088877 0.286364
  3_1024      2048       1024  0.193595 0.255544
   3_512      2048        512  0.035564 0.284100
  4_1024      2048        210  0.093439 0.284564
   4_512      2048        184 -0.070000 0.314527

Saved summary CSV to: ml_runs_dimred_AtomPairs_Gen/summary_results_AtomPairs_Gen.csv


In [ ]:
# chirality included (new feature introduced in API)

import numpy as np
import pandas as pd
from pathlib import Path

from rdkit import Chem, DataStructs
from rdkit.Chem import rdFingerprintGenerator as rfg  # AtomPairGenerator

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import r2_score

# -----------------------------
# CONFIG
# -----------------------------
N_BITS = 2048
INCLUDE_CHIRALITY = True       # << turn chirality on/off here
MIN_DISTANCE = 1
MAX_DISTANCE = 30

N_SPLITS = 200
TEST_SIZE = 0.20
SEEDS = np.arange(N_SPLITS)    # 0..199

RARE_MIN = 3
UBIQ_GT = 60

CASES = ["1", "2", "3_1024", "3_512", "4_1024", "4_512"]

INPUT_XLSX = "HOMO-LUMO-energies.xlsx"   # must have columns: Smiles, dFF
OUT_DIR = Path("ml_runs_dimred_AtomPairs_Chiral")
OUT_DIR.mkdir(parents=True, exist_ok=True)
SUMMARY_CSV = OUT_DIR / "summary_results_AtomPairs_Chiral.csv"

# -----------------------------
# UTILITIES
# -----------------------------
def smiles_to_atompairs(smiles_list, nBits=N_BITS,
                        minDistance=MIN_DISTANCE, maxDistance=MAX_DISTANCE,
                        includeChirality=INCLUDE_CHIRALITY):
    """AtomPairGenerator -> numpy array (N, nBits) with optional chirality."""
    gen = rfg.GetAtomPairGenerator(
        fpSize=nBits,
        minDistance=minDistance,
        maxDistance=maxDistance,
        includeChirality=includeChirality
    )
    mols = [Chem.MolFromSmiles(s) for s in smiles_list]
    X = np.zeros((len(mols), nBits), dtype=np.uint8)
    for i, m in enumerate(mols):
        if m is None:
            continue
        fp = gen.GetFingerprint(m)
        DataStructs.ConvertToNumpyArray(fp, X[i])
    return X

def bit_presence(X): return (X > 0).astype(np.uint8)
def compute_freq_initial_only(X_init): return bit_presence(X_init).sum(axis=0)

def select_cols_by_freq(freq, rule):
    if rule == "case1":
        return (freq > 0)
    elif rule == "case2":
        return (freq > 0) & (freq >= RARE_MIN) & (freq <= UBIQ_GT)
    else:
        raise ValueError("rule must be 'case1' or 'case2'")

def apply_mask(X, mask): return X[:, mask]
def build_fold_map(orig_dim, new_dim): return np.array([j % new_dim for j in range(orig_dim)], dtype=int)

def fold_binary_OR(X, idx_map):
    new_dim = idx_map.max() + 1
    out = np.zeros((X.shape[0], new_dim), dtype=np.uint8)
    for tgt in range(new_dim):
        cols = (idx_map == tgt)
        if np.any(cols):
            out[:, tgt] = np.max(X[:, cols], axis=1)
    return out

def transform_case_initial_only(X_init, case):
    meta = {"orig_dim": X_init.shape[1]}
    if case in ("1", "2"):
        freq = compute_freq_initial_only(X_init)
        rule = "case1" if case == "1" else "case2"
        mask = select_cols_by_freq(freq, rule)
        X_out = apply_mask(X_init, mask)
        meta.update({"rule": rule, "final_dim": int(mask.sum()), "dropped": int((~mask).sum())})
        return X_out, meta
    elif case.startswith("3_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(X_init.shape[1], new_dim)
        X_out = fold_binary_OR(X_init, idx_map)
        meta.update({"folded_to": new_dim, "final_dim": new_dim, "dropped": 0})
        return X_out, meta
    elif case.startswith("4_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(X_init.shape[1], new_dim)
        X_fold = fold_binary_OR(X_init, idx_map)
        freq_fold = compute_freq_initial_only(X_fold)
        mask = select_cols_by_freq(freq_fold, "case2")
        X_out = apply_mask(X_fold, mask)
        meta.update({"folded_to": new_dim, "rule": "case2_after_fold",
                     "final_dim": int(mask.sum()), "dropped_after_fold": int((~mask).sum())})
        return X_out, meta
    else:
        raise ValueError(f"Unknown case: {case}")

# -----------------------------
# LOAD DATA
# -----------------------------
df = pd.read_excel(INPUT_XLSX)
smiles = df["Smiles"].astype(str).tolist()
y = df["dFF"].values

X0 = smiles_to_atompairs(smiles)  # (N, 2048) with chirality

# -----------------------------
# BASELINE (no reduction)
# -----------------------------
r2s = []
baseline_txt = OUT_DIR / "r2_SVRrbf_baseline.txt"
with open(baseline_txt, "w") as f:
    f.write("# seed\tR2\n")
    for seed in SEEDS:
        X_tr, X_te, y_tr, y_te = train_test_split(X0, y, test_size=TEST_SIZE, random_state=int(seed))
        model = SVR(kernel="rbf")
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_te)
        r2 = r2_score(y_te, y_pred)
        r2s.append(r2)
        f.write(f"{int(seed)}\t{r2:.6f}\n")
baseline_mean, baseline_std = float(np.mean(r2s)), float(np.std(r2s))
print(f"AtomPairs (chiral) Baseline (2048 bits) -> Mean R2: {baseline_mean:.6f}, Std: {baseline_std:.6f}")

# -----------------------------
# RUN CASES 1–4
# -----------------------------
summary_rows = [{
    "Case": "baseline",
    "Orig_Dim": X0.shape[1],
    "Final_Dim": X0.shape[1],
    "R2_mean": baseline_mean,
    "R2_std": baseline_std,
    "Notes": "Full 2048-bit AtomPairs (includeChirality=True)"
}]

for case in CASES:
    X_case, meta = transform_case_initial_only(X0, case)
    r2s = []
    out_txt = OUT_DIR / f"r2_SVRrbf_case{case}.txt"
    with open(out_txt, "w") as f:
        f.write("# seed\tR2\n")
        for seed in SEEDS:
            X_tr, X_te, y_tr, y_te = train_test_split(X_case, y, test_size=TEST_SIZE, random_state=int(seed))
            model = SVR(kernel="rbf")
            model.fit(X_tr, y_tr)
            y_pred = model.predict(X_te)
            r2 = r2_score(y_te, y_pred)
            r2s.append(r2)
            f.write(f"{int(seed)}\t{r2:.6f}\n")
    summary_rows.append({
        "Case": case,
        "Orig_Dim": meta.get("orig_dim", X0.shape[1]),
        "Final_Dim": meta.get("final_dim", X_case.shape[1]),
        "R2_mean": float(np.mean(r2s)),
        "R2_std": float(np.std(r2s)),
        "Notes": {k:v for k,v in meta.items() if k not in ["orig_dim","final_dim"]}
    })

# -----------------------------
# SAVE SUMMARY
# -----------------------------
summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv(SUMMARY_CSV, index=False)

print("\n=== AtomPairs (chiral) SVR (RBF) Summary over 200 splits ===")
print(summary_df[["Case","Orig_Dim","Final_Dim","R2_mean","R2_std"]].to_string(index=False))
print(f"\nSaved summary CSV to: {SUMMARY_CSV}")


AtomPairs (chiral) Baseline (2048 bits) -> Mean R2: 0.140245, Std: 0.269280

=== AtomPairs (chiral) SVR (RBF) Summary over 200 splits ===
    Case  Orig_Dim  Final_Dim  R2_mean   R2_std
baseline      2048       2048 0.140245 0.269280
       1      2048        499 0.131316 0.271080
       2      2048        217 0.183963 0.263411
  3_1024      2048       1024 0.158794 0.268067
   3_512      2048        512 0.174627 0.263350
  4_1024      2048        207 0.196562 0.268166
   4_512      2048        179 0.192094 0.269671

Saved summary CSV to: ml_runs_dimred_AtomPairs_Chiral/summary_results_AtomPairs_Chiral.csv


# Torsion

In [ ]:
# If in Colab (once per runtime):
# !pip -q install rdkit openpyxl

import numpy as np
import pandas as pd
from pathlib import Path

from rdkit import Chem, DataStructs
from rdkit.Chem import rdFingerprintGenerator as rfg  # TopologicalTorsionGenerator

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import r2_score

# -----------------------------
# CONFIG
# -----------------------------
N_BITS = 2048
INCLUDE_CHIRALITY = False   # set True to encode stereochemistry
TORSION_SIZE = 4            # classic topological torsion uses 4-atom fragments

N_SPLITS = 200
TEST_SIZE = 0.20
SEEDS = np.arange(N_SPLITS)  # 0..199

# thresholds per your ~63-molecule set
RARE_MIN = 3
UBIQ_GT  = 60

# Cases:
# 1: drop never-seen bits
# 2: drop never-seen, rare (<3), ubiquitous (>60)
# 3_k: fold 2048 -> k by modulo (no dropping)
# 4_k: fold, then Case-2 dropping on folded space
CASES = ["1", "2", "3_1024", "3_512", "4_1024", "4_512"]

INPUT_XLSX = "HOMO-LUMO-energies.xlsx"   # must have columns: Smiles, dFF
OUT_DIR = Path("ml_runs_dimred_Torsions")
OUT_DIR.mkdir(parents=True, exist_ok=True)
SUMMARY_CSV = OUT_DIR / "summary_results_Torsions.csv"

# -----------------------------
# UTILITIES
# -----------------------------
def smiles_to_torsions(smiles_list, nBits=N_BITS, torsionSize=TORSION_SIZE, includeChirality=INCLUDE_CHIRALITY):
    """
    RDKit TopologicalTorsionGenerator -> numpy (N, nBits).
    torsionSize=4 is the standard; includeChirality toggles stereo.
    """
    gen = rfg.GetTopologicalTorsionGenerator(
        fpSize=nBits,
        torsionAtomCount=torsionSize,     # sometimes 'atomCount' or 'torsionAtomCount' across RDKit versions
        includeChirality=includeChirality
    )
    mols = [Chem.MolFromSmiles(s) for s in smiles_list]
    X = np.zeros((len(mols), nBits), dtype=np.uint8)
    for i, m in enumerate(mols):
        if m is None:
            continue
        fp = gen.GetFingerprint(m)                 # ExplicitBitVect
        DataStructs.ConvertToNumpyArray(fp, X[i])  # fill row in-place
    return X

def bit_presence(X): return (X > 0).astype(np.uint8)

def compute_freq_initial_only(X_init):
    # count per-bit presence across molecules
    return bit_presence(X_init).sum(axis=0)

def select_cols_by_freq(freq, rule):
    if rule == "case1":        # drop never-seen
        return (freq > 0)
    elif rule == "case2":      # drop never-seen, rare (<3), ubiquitous (>60)
        return (freq > 0) & (freq >= RARE_MIN) & (freq <= UBIQ_GT)
    else:
        raise ValueError("rule must be 'case1' or 'case2'")

def apply_mask(X, mask): return X[:, mask]

def build_fold_map(orig_dim, new_dim):
    # modulo hashing: original column j -> j % new_dim
    return np.array([j % new_dim for j in range(orig_dim)], dtype=int)

def fold_binary_OR(X, idx_map):
    """Fold columns into new_dim buckets using OR aggregation."""
    new_dim = int(idx_map.max()) + 1
    out = np.zeros((X.shape[0], new_dim), dtype=np.uint8)
    for tgt in range(new_dim):
        cols = (idx_map == tgt)
        if np.any(cols):
            out[:, tgt] = np.max(X[:, cols], axis=1)
    return out

def transform_case_initial_only(X_init, case):
    """
    Case '1': drop never-seen bits
    Case '2': drop never-seen, rare (<3), ubiquitous (>60)
    Case '3_k': fold to k (no dropping)
    Case '4_k': fold to k, then Case-2 dropping on folded space
    """
    meta = {"orig_dim": X_init.shape[1]}
    if case in ("1", "2"):
        freq = compute_freq_initial_only(X_init)
        rule = "case1" if case == "1" else "case2"
        mask = select_cols_by_freq(freq, rule)
        X_out = apply_mask(X_init, mask)
        meta.update({"rule": rule, "final_dim": int(mask.sum()), "dropped": int((~mask).sum())})
        return X_out, meta

    elif case.startswith("3_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(X_init.shape[1], new_dim)
        X_out = fold_binary_OR(X_init, idx_map)
        meta.update({"folded_to": new_dim, "final_dim": new_dim, "dropped": 0})
        return X_out, meta

    elif case.startswith("4_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(X_init.shape[1], new_dim)
        X_fold = fold_binary_OR(X_init, idx_map)
        freq_fold = compute_freq_initial_only(X_fold)
        mask = select_cols_by_freq(freq_fold, "case2")
        X_out = apply_mask(X_fold, mask)
        meta.update({"folded_to": new_dim, "rule": "case2_after_fold",
                     "final_dim": int(mask.sum()), "dropped_after_fold": int((~mask).sum())})
        return X_out, meta

    else:
        raise ValueError(f"Unknown case: {case}")

# -----------------------------
# LOAD DATA
# -----------------------------
df = pd.read_excel(INPUT_XLSX)
smiles = df["Smiles"].astype(str).tolist()
y = df["dFF"].values

X0 = smiles_to_torsions(smiles)  # (N, 2048) topological torsions

# -----------------------------
# BASELINE (no reduction)
# -----------------------------
r2s = []
baseline_txt = OUT_DIR / "r2_SVRrbf_baseline.txt"
with open(baseline_txt, "w") as f:
    f.write("# seed\tR2\n")
    for seed in SEEDS:
        X_tr, X_te, y_tr, y_te = train_test_split(X0, y, test_size=TEST_SIZE, random_state=int(seed))
        model = SVR(kernel="rbf")
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_te)
        r2 = r2_score(y_te, y_pred)
        r2s.append(r2)
        f.write(f"{int(seed)}\t{r2:.6f}\n")
baseline_mean, baseline_std = float(np.mean(r2s)), float(np.std(r2s))
print(f"Torsions Baseline (2048 bits) -> Mean R2: {baseline_mean:.6f}, Std: {baseline_std:.6f}")

# -----------------------------
# RUN CASES 1–4
# -----------------------------
summary_rows = [{
    "Case": "baseline",
    "Orig_Dim": X0.shape[1],
    "Final_Dim": X0.shape[1],
    "R2_mean": baseline_mean,
    "R2_std": baseline_std,
    "Notes": f"Full 2048-bit Topological Torsions (chirality={INCLUDE_CHIRALITY})"
}]

for case in CASES:
    X_case, meta = transform_case_initial_only(X0, case)
    r2s = []
    out_txt = OUT_DIR / f"r2_SVRrbf_case{case}.txt"
    with open(out_txt, "w") as f:
        f.write("# seed\tR2\n")
        for seed in SEEDS:
            X_tr, X_te, y_tr, y_te = train_test_split(X_case, y, test_size=TEST_SIZE, random_state=int(seed))
            model = SVR(kernel="rbf")
            model.fit(X_tr, y_tr)
            y_pred = model.predict(X_te)
            r2 = r2_score(y_te, y_pred)
            r2s.append(r2)
            f.write(f"{int(seed)}\t{r2:.6f}\n")
    summary_rows.append({
        "Case": case,
        "Orig_Dim": meta.get("orig_dim", X0.shape[1]),
        "Final_Dim": meta.get("final_dim", X_case.shape[1]),
        "R2_mean": float(np.mean(r2s)),
        "R2_std": float(np.std(r2s)),
        "Notes": {k:v for k,v in meta.items() if k not in ["orig_dim","final_dim"]}
    })

# -----------------------------
# SAVE SUMMARY
# -----------------------------
summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv(SUMMARY_CSV, index=False)

print("\n=== Topological Torsions SVR (RBF) Summary over 200 splits ===")
print(summary_df[["Case","Orig_Dim","Final_Dim","R2_mean","R2_std"]].to_string(index=False))
print(f"\nSaved summary CSV to: {SUMMARY_CSV}")


Torsions Baseline (2048 bits) -> Mean R2: 0.211189, Std: 0.256045

=== Topological Torsions SVR (RBF) Summary over 200 splits ===
    Case  Orig_Dim  Final_Dim  R2_mean   R2_std
baseline      2048       2048 0.211189 0.256045
       1      2048        250 0.203758 0.256504
       2      2048         73 0.273915 0.262173
  3_1024      2048       1024 0.221129 0.256251
   3_512      2048        512 0.232846 0.255752
  4_1024      2048         78 0.294915 0.245903
   4_512      2048         79 0.267639 0.260383

Saved summary CSV to: ml_runs_dimred_Torsions/summary_results_Torsions.csv


In [ ]:
# If in Colab (once per runtime):
# !pip -q install rdkit openpyxl

import numpy as np
import pandas as pd
from pathlib import Path

from rdkit import Chem, DataStructs
from rdkit.Chem import rdFingerprintGenerator as rfg  # TopologicalTorsionGenerator

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import r2_score

# -----------------------------
# CONFIG
# -----------------------------
N_BITS = 2048
INCLUDE_CHIRALITY = False   # set True to encode stereochemistry
TORSION_SIZE = 4            # classic topological torsion uses 4-atom fragments

N_SPLITS = 200
TEST_SIZE = 0.20
SEEDS = np.arange(N_SPLITS)  # 0..199

# thresholds per your ~63-molecule set
RARE_MIN = 3
UBIQ_GT  = 60

# Cases:
# 1: drop never-seen bits
# 2: drop never-seen, rare (<3), ubiquitous (>60)
# 3_k: fold 2048 -> k by modulo (no dropping)
# 4_k: fold, then Case-2 dropping on folded space
CASES = ["1", "2", "3_1024", "3_512", "4_1024", "4_512"]

INPUT_XLSX = "HOMO-LUMO-energies.xlsx"   # must have columns: Smiles, dFF
OUT_DIR = Path("ml_runs_dimred_Torsions")
OUT_DIR.mkdir(parents=True, exist_ok=True)
SUMMARY_CSV = OUT_DIR / "summary_results_Torsions.csv"

# -----------------------------
# UTILITIES
# -----------------------------
def smiles_to_torsions(smiles_list, nBits=N_BITS, torsionSize=TORSION_SIZE, includeChirality=INCLUDE_CHIRALITY):
    """
    RDKit TopologicalTorsionGenerator -> numpy (N, nBits).
    torsionSize=4 is the standard; includeChirality toggles stereo.
    """
    gen = rfg.GetTopologicalTorsionGenerator(
        fpSize=nBits,
        torsionAtomCount=torsionSize,     # sometimes 'atomCount' or 'torsionAtomCount' across RDKit versions
        includeChirality=includeChirality
    )
    mols = [Chem.MolFromSmiles(s) for s in smiles_list]
    X = np.zeros((len(mols), nBits), dtype=np.uint8)
    for i, m in enumerate(mols):
        if m is None:
            continue
        fp = gen.GetFingerprint(m)                 # ExplicitBitVect
        DataStructs.ConvertToNumpyArray(fp, X[i])  # fill row in-place
    return X

def bit_presence(X): return (X > 0).astype(np.uint8)

def compute_freq_initial_only(X_init):
    # count per-bit presence across molecules
    return bit_presence(X_init).sum(axis=0)

def select_cols_by_freq(freq, rule):
    if rule == "case1":        # drop never-seen
        return (freq > 0)
    elif rule == "case2":      # drop never-seen, rare (<3), ubiquitous (>60)
        return (freq > 0) & (freq >= RARE_MIN) & (freq <= UBIQ_GT)
    else:
        raise ValueError("rule must be 'case1' or 'case2'")

def apply_mask(X, mask): return X[:, mask]

def build_fold_map(orig_dim, new_dim):
    # modulo hashing: original column j -> j % new_dim
    return np.array([j % new_dim for j in range(orig_dim)], dtype=int)

def fold_binary_OR(X, idx_map):
    """Fold columns into new_dim buckets using OR aggregation."""
    new_dim = int(idx_map.max()) + 1
    out = np.zeros((X.shape[0], new_dim), dtype=np.uint8)
    for tgt in range(new_dim):
        cols = (idx_map == tgt)
        if np.any(cols):
            out[:, tgt] = np.max(X[:, cols], axis=1)
    return out

def transform_case_initial_only(X_init, case):
    """
    Case '1': drop never-seen bits
    Case '2': drop never-seen, rare (<3), ubiquitous (>60)
    Case '3_k': fold to k (no dropping)
    Case '4_k': fold to k, then Case-2 dropping on folded space
    """
    meta = {"orig_dim": X_init.shape[1]}
    if case in ("1", "2"):
        freq = compute_freq_initial_only(X_init)
        rule = "case1" if case == "1" else "case2"
        mask = select_cols_by_freq(freq, rule)
        X_out = apply_mask(X_init, mask)
        meta.update({"rule": rule, "final_dim": int(mask.sum()), "dropped": int((~mask).sum())})
        return X_out, meta

    elif case.startswith("3_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(X_init.shape[1], new_dim)
        X_out = fold_binary_OR(X_init, idx_map)
        meta.update({"folded_to": new_dim, "final_dim": new_dim, "dropped": 0})
        return X_out, meta

    elif case.startswith("4_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(X_init.shape[1], new_dim)
        X_fold = fold_binary_OR(X_init, idx_map)
        freq_fold = compute_freq_initial_only(X_fold)
        mask = select_cols_by_freq(freq_fold, "case2")
        X_out = apply_mask(X_fold, mask)
        meta.update({"folded_to": new_dim, "rule": "case2_after_fold",
                     "final_dim": int(mask.sum()), "dropped_after_fold": int((~mask).sum())})
        return X_out, meta

    else:
        raise ValueError(f"Unknown case: {case}")

# -----------------------------
# LOAD DATA
# -----------------------------
df = pd.read_excel(INPUT_XLSX)
smiles = df["Smiles"].astype(str).tolist()
y = df["dFF"].values

X0 = smiles_to_torsions(smiles)  # (N, 2048) topological torsions

# -----------------------------
# BASELINE (no reduction)
# -----------------------------
r2s = []
baseline_txt = OUT_DIR / "r2_SVRlinear_baseline.txt"
with open(baseline_txt, "w") as f:
    f.write("# seed\tR2\n")
    for seed in SEEDS:
        X_tr, X_te, y_tr, y_te = train_test_split(X0, y, test_size=TEST_SIZE, random_state=int(seed))
        model = SVR(kernel="linear")
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_te)
        r2 = r2_score(y_te, y_pred)
        r2s.append(r2)
        f.write(f"{int(seed)}\t{r2:.6f}\n")
baseline_mean, baseline_std = float(np.mean(r2s)), float(np.std(r2s))
print(f"Torsions Baseline (2048 bits) -> Mean R2: {baseline_mean:.6f}, Std: {baseline_std:.6f}")

# -----------------------------
# RUN CASES 1–4
# -----------------------------
summary_rows = [{
    "Case": "baseline",
    "Orig_Dim": X0.shape[1],
    "Final_Dim": X0.shape[1],
    "R2_mean": baseline_mean,
    "R2_std": baseline_std,
    "Notes": f"Full 2048-bit Topological Torsions (chirality={INCLUDE_CHIRALITY})"
}]

for case in CASES:
    X_case, meta = transform_case_initial_only(X0, case)
    r2s = []
    out_txt = OUT_DIR / f"r2_SVRlinear_case{case}.txt"
    with open(out_txt, "w") as f:
        f.write("# seed\tR2\n")
        for seed in SEEDS:
            X_tr, X_te, y_tr, y_te = train_test_split(X_case, y, test_size=TEST_SIZE, random_state=int(seed))
            model = SVR(kernel="linear")
            model.fit(X_tr, y_tr)
            y_pred = model.predict(X_te)
            r2 = r2_score(y_te, y_pred)
            r2s.append(r2)
            f.write(f"{int(seed)}\t{r2:.6f}\n")
    summary_rows.append({
        "Case": case,
        "Orig_Dim": meta.get("orig_dim", X0.shape[1]),
        "Final_Dim": meta.get("final_dim", X_case.shape[1]),
        "R2_mean": float(np.mean(r2s)),
        "R2_std": float(np.std(r2s)),
        "Notes": {k:v for k,v in meta.items() if k not in ["orig_dim","final_dim"]}
    })

# -----------------------------
# SAVE SUMMARY
# -----------------------------
summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv(SUMMARY_CSV, index=False)

print("\n=== Topological Torsions SVR (linear) Summary over 200 splits ===")
print(summary_df[["Case","Orig_Dim","Final_Dim","R2_mean","R2_std"]].to_string(index=False))
print(f"\nSaved summary CSV to: {SUMMARY_CSV}")


Torsions Baseline (2048 bits) -> Mean R2: 0.359225, Std: 0.354232

=== Topological Torsions SVR (linear) Summary over 200 splits ===
    Case  Orig_Dim  Final_Dim   R2_mean   R2_std
baseline      2048       2048  0.359225 0.354232
       1      2048        250  0.359225 0.354232
       2      2048         73 -0.080918 0.658533
  3_1024      2048       1024  0.276438 0.364637
   3_512      2048        512  0.247835 0.374568
  4_1024      2048         78  0.180351 0.478825
   4_512      2048         79  0.150740 0.475900

Saved summary CSV to: ml_runs_dimred_Torsions/summary_results_Torsions.csv


In [ ]:
# If in Colab (once per runtime):
# !pip -q install rdkit openpyxl

import numpy as np
import pandas as pd
from pathlib import Path

from rdkit import Chem, DataStructs
from rdkit.Chem import rdFingerprintGenerator as rfg  # TopologicalTorsionGenerator

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import r2_score

# -----------------------------
# CONFIG
# -----------------------------
N_BITS = 2048
INCLUDE_CHIRALITY = False   # set True to encode stereochemistry
TORSION_SIZE = 4            # classic topological torsion uses 4-atom fragments

N_SPLITS = 200
TEST_SIZE = 0.20
SEEDS = np.arange(N_SPLITS)  # 0..199

# thresholds per your ~63-molecule set
RARE_MIN = 3
UBIQ_GT  = 60

# Cases:
# 1: drop never-seen bits
# 2: drop never-seen, rare (<3), ubiquitous (>60)
# 3_k: fold 2048 -> k by modulo (no dropping)
# 4_k: fold, then Case-2 dropping on folded space
CASES = ["1", "2", "3_1024", "3_512", "4_1024", "4_512"]

INPUT_XLSX = "HOMO-LUMO-energies.xlsx"   # must have columns: Smiles, dFF
OUT_DIR = Path("ml_runs_dimred_Torsions")
OUT_DIR.mkdir(parents=True, exist_ok=True)
SUMMARY_CSV = OUT_DIR / "summary_results_Torsions.csv"

# -----------------------------
# UTILITIES
# -----------------------------
def smiles_to_torsions(smiles_list, nBits=N_BITS, torsionSize=TORSION_SIZE, includeChirality=INCLUDE_CHIRALITY):
    """
    RDKit TopologicalTorsionGenerator -> numpy (N, nBits).
    torsionSize=4 is the standard; includeChirality toggles stereo.
    """
    gen = rfg.GetTopologicalTorsionGenerator(
        fpSize=nBits,
        torsionAtomCount=torsionSize,     # sometimes 'atomCount' or 'torsionAtomCount' across RDKit versions
        includeChirality=includeChirality
    )
    mols = [Chem.MolFromSmiles(s) for s in smiles_list]
    X = np.zeros((len(mols), nBits), dtype=np.uint8)
    for i, m in enumerate(mols):
        if m is None:
            continue
        fp = gen.GetFingerprint(m)                 # ExplicitBitVect
        DataStructs.ConvertToNumpyArray(fp, X[i])  # fill row in-place
    return X

def bit_presence(X): return (X > 0).astype(np.uint8)

def compute_freq_initial_only(X_init):
    # count per-bit presence across molecules
    return bit_presence(X_init).sum(axis=0)

def select_cols_by_freq(freq, rule):
    if rule == "case1":        # drop never-seen
        return (freq > 0)
    elif rule == "case2":      # drop never-seen, rare (<3), ubiquitous (>60)
        return (freq > 0) & (freq >= RARE_MIN) & (freq <= UBIQ_GT)
    else:
        raise ValueError("rule must be 'case1' or 'case2'")

def apply_mask(X, mask): return X[:, mask]

def build_fold_map(orig_dim, new_dim):
    # modulo hashing: original column j -> j % new_dim
    return np.array([j % new_dim for j in range(orig_dim)], dtype=int)

def fold_binary_OR(X, idx_map):
    """Fold columns into new_dim buckets using OR aggregation."""
    new_dim = int(idx_map.max()) + 1
    out = np.zeros((X.shape[0], new_dim), dtype=np.uint8)
    for tgt in range(new_dim):
        cols = (idx_map == tgt)
        if np.any(cols):
            out[:, tgt] = np.max(X[:, cols], axis=1)
    return out

def transform_case_initial_only(X_init, case):
    """
    Case '1': drop never-seen bits
    Case '2': drop never-seen, rare (<3), ubiquitous (>60)
    Case '3_k': fold to k (no dropping)
    Case '4_k': fold to k, then Case-2 dropping on folded space
    """
    meta = {"orig_dim": X_init.shape[1]}
    if case in ("1", "2"):
        freq = compute_freq_initial_only(X_init)
        rule = "case1" if case == "1" else "case2"
        mask = select_cols_by_freq(freq, rule)
        X_out = apply_mask(X_init, mask)
        meta.update({"rule": rule, "final_dim": int(mask.sum()), "dropped": int((~mask).sum())})
        return X_out, meta

    elif case.startswith("3_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(X_init.shape[1], new_dim)
        X_out = fold_binary_OR(X_init, idx_map)
        meta.update({"folded_to": new_dim, "final_dim": new_dim, "dropped": 0})
        return X_out, meta

    elif case.startswith("4_"):
        new_dim = int(case.split("_")[1])
        idx_map = build_fold_map(X_init.shape[1], new_dim)
        X_fold = fold_binary_OR(X_init, idx_map)
        freq_fold = compute_freq_initial_only(X_fold)
        mask = select_cols_by_freq(freq_fold, "case2")
        X_out = apply_mask(X_fold, mask)
        meta.update({"folded_to": new_dim, "rule": "case2_after_fold",
                     "final_dim": int(mask.sum()), "dropped_after_fold": int((~mask).sum())})
        return X_out, meta

    else:
        raise ValueError(f"Unknown case: {case}")

# -----------------------------
# LOAD DATA
# -----------------------------
df = pd.read_excel(INPUT_XLSX)
smiles = df["Smiles"].astype(str).tolist()
y = df["dFF"].values

X0 = smiles_to_torsions(smiles)  # (N, 2048) topological torsions

# -----------------------------
# BASELINE (no reduction)
# -----------------------------
r2s = []
baseline_txt = OUT_DIR / "r2_SVRsigmoid_baseline.txt"
with open(baseline_txt, "w") as f:
    f.write("# seed\tR2\n")
    for seed in SEEDS:
        X_tr, X_te, y_tr, y_te = train_test_split(X0, y, test_size=TEST_SIZE, random_state=int(seed))
        model = SVR(kernel="sigmoid")
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_te)
        r2 = r2_score(y_te, y_pred)
        r2s.append(r2)
        f.write(f"{int(seed)}\t{r2:.6f}\n")
baseline_mean, baseline_std = float(np.mean(r2s)), float(np.std(r2s))
print(f"Torsions Baseline (2048 bits) -> Mean R2: {baseline_mean:.6f}, Std: {baseline_std:.6f}")

# -----------------------------
# RUN CASES 1–4
# -----------------------------
summary_rows = [{
    "Case": "baseline",
    "Orig_Dim": X0.shape[1],
    "Final_Dim": X0.shape[1],
    "R2_mean": baseline_mean,
    "R2_std": baseline_std,
    "Notes": f"Full 2048-bit Topological Torsions (chirality={INCLUDE_CHIRALITY})"
}]

for case in CASES:
    X_case, meta = transform_case_initial_only(X0, case)
    r2s = []
    out_txt = OUT_DIR / f"r2_SVRsigmoid_case{case}.txt"
    with open(out_txt, "w") as f:
        f.write("# seed\tR2\n")
        for seed in SEEDS:
            X_tr, X_te, y_tr, y_te = train_test_split(X_case, y, test_size=TEST_SIZE, random_state=int(seed))
            model = SVR(kernel="sigmoid")
            model.fit(X_tr, y_tr)
            y_pred = model.predict(X_te)
            r2 = r2_score(y_te, y_pred)
            r2s.append(r2)
            f.write(f"{int(seed)}\t{r2:.6f}\n")
    summary_rows.append({
        "Case": case,
        "Orig_Dim": meta.get("orig_dim", X0.shape[1]),
        "Final_Dim": meta.get("final_dim", X_case.shape[1]),
        "R2_mean": float(np.mean(r2s)),
        "R2_std": float(np.std(r2s)),
        "Notes": {k:v for k,v in meta.items() if k not in ["orig_dim","final_dim"]}
    })

# -----------------------------
# SAVE SUMMARY
# -----------------------------
summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv(SUMMARY_CSV, index=False)

print("\n=== Topological Torsions SVR (sigmoid) Summary over 200 splits ===")
print(summary_df[["Case","Orig_Dim","Final_Dim","R2_mean","R2_std"]].to_string(index=False))
print(f"\nSaved summary CSV to: {SUMMARY_CSV}")


Torsions Baseline (2048 bits) -> Mean R2: 0.283777, Std: 0.298675

=== Topological Torsions SVR (sigmoid) Summary over 200 splits ===
    Case  Orig_Dim  Final_Dim  R2_mean   R2_std
baseline      2048       2048 0.283777 0.298675
       1      2048        250 0.273889 0.303755
       2      2048         73 0.104615 0.385871
  3_1024      2048       1024 0.311751 0.302179
   3_512      2048        512 0.319093 0.285032
  4_1024      2048         78 0.207979 0.350605
   4_512      2048         79 0.162042 0.337238

Saved summary CSV to: ml_runs_dimred_Torsions/summary_results_Torsions.csv
